# **PROYECTO SCHUTZ EN PYTHON PARA CREACIÓN DE DASHBOARD 1 - PARTE 1**

## 1.- IMPORTACIÓN DE LIBRERÍAS PARA EL PROYECTO

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO
from itertools import zip_longest
import io
import requests
from io import BytesIO
import pandas as pd
import chardet
import re
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from gspread_dataframe import set_with_dataframe
from urllib.request import FancyURLopener
from urllib.request import urlopen
import urllib.request
import xml.etree.ElementTree as ET
import numpy as np
import datetime
import ssl
import warnings
from datetime import datetime, timedelta


In [2]:
pd.options.display.max_rows = None
pd.options.display.float_format = '{:.2f}'.format

warnings.filterwarnings("ignore", category=FutureWarning, message="Inferring datetime64.*")

## 2.- LECTURA DE DATAFRAMES

### 2.1 LECTURA "LIBRO MAYOR"

In [3]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\Libro%20Mayor%202023.txt'

# Descargar el archivo XML desde la URL
with urllib.request.urlopen(url) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_libro_mayor = pd.DataFrame(data)

# Imprimir el dataframe
df_libro_mayor.sample(5)

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia
902,4101010-000,2023,1,2023-01-04T00:00:00-03:00,CIN,74,"Brocha 2 1/2""",P-1682,I17440408,1000,0,48723
53,3101030-000,2023,1,2023-01-19T00:00:00-03:00,CVT,676,Segun contrato N° ECBI-299-49 DE FECHA 11-02-2...,P-1695,84102200-9,0,0,13319
6195,4101010-000,2023,6,2023-06-05T00:00:00-04:00,CIN,106,Conector Jumper FO Duplex LC/LC MM OM3 1,P-1727,C02041975,39960,0,51535
3407,4101010-000,2023,3,2023-03-31T00:00:00-03:00,CIN,1578,Cinta Aisladora 3M Temflex 1500,P-1703,I16430107,2935,0,50122
3803,4101010-000,2023,4,2023-04-10T00:00:00-03:00,CIN,182,"Disco corte 4 1/2"" metal",P-1680,I17440838,1895,0,50276


In [4]:
df_libro_mayor.to_csv('archivos_respaldo/df_libro_mayor_1.csv', index = False)

### 2.2 LECTURA "DESPACHOS REALIZADOS"

In [5]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\Despachos%20Realizados.txt'

# Descargar el archivo XML desde la URL
with urllib.request.urlopen(url) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_desp_realizados = pd.DataFrame(data)

# Imprimir el dataframe
df_desp_realizados.sample(5)

,COD_PROYECTO,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL
19130,P-Proyectos,VCO,2023-04-17T00:00:00-03:00,50487,None,H21570765,1,Casco Seguridad Simple Azul,UN,1832,1832
10955,P-1682,VCO,2023-01-10T00:00:00-03:00,48860,1035,I17440479,1,Broca SDS concreto 10mm,UN,2458,2458
17483,P-1704,VCO,2022-07-21T00:00:00-04:00,46374,1110,I15410493,200,"Golilla plana 3/8"" zincada",UN,14,2800
8178,P-1701,VCO,2022-08-02T00:00:00-04:00,46553,1023,I17441729,100,"Tornillo Roscalata 6"" x 3""",UN,22,2200
16666,P-1691,VCO,2022-09-01T00:00:00-03:00,46954,1037,T14400271,1,Spray prueba detector de humo,UN,11990,11990


In [6]:
df_desp_realizados.to_csv('archivos_respaldo/df_desp_realizados_1.csv', index = False)

### 2.3 LECTURA "CODIGOS DE PROYECTO"

In [7]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\codigos%20de%20proyectos.txt'

# Descargar el archivo XML desde la URL
with urllib.request.urlopen(url) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_cod_proyecto = pd.DataFrame(data)

# Imprimir el dataframe
df_cod_proyecto.sample(5)

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE
190,P-1680,Nueva Uno,Constructora Tecsa S.A.
166,P-1517,Estacionamientos Pedro de Valdivia,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO
19,P-1204,Vista Los Andes,BERSA KENNEDY S.A
203,P-1705,Los Clarines,CONSTRUCTORA INARCO S.A.
143,P-1583,San Ignacio,CONSTRUCTORA ARMAS LTDA


In [8]:
df_cod_proyecto.to_csv('archivos_respaldo/df_cod_proyecto_1.csv', index = False)

### 2.4 LECTURA "PROYECTADOS"

In [9]:
# Se crea un filtro para obtener todos aquellos que tengan valores hasta el MES anterior al mes en curso
import datetime

# Obtener el mes actual de acuerdo a la fecha actual, y el resultado se convierte en el nombre del MES para luego buscar aquel nombre en el archivo.
mes_actual = datetime.date.today().month

print(mes_actual)

if mes_actual == 1:
    mes_actual2 = "ENERO"
elif mes_actual == 2:
    mes_actual2 = "FEBRERO"
elif mes_actual == 3:
    mes_actual2 = "MARZO"
elif mes_actual == 4:
    mes_actual2 = "ABRIL"
elif mes_actual == 5:
    mes_actual2 = "MAYO"
elif mes_actual == 6:
    mes_actual2 = "JUNIO"
elif mes_actual == 7:
    mes_actual2 = "JULIO"
elif mes_actual == 8:
    mes_actual2 = "AGOSTO"
elif mes_actual == 9:
    mes_actual2 = "SEPTIEMBRE"
elif mes_actual == 10:
    mes_actual2 = "OCTUBRE"
elif mes_actual == 11:
    mes_actual2 = "NOVIEMBRE"
else:
    mes_actual2 = "DICIEMBRE"

print(mes_actual2)

6
JUNIO


In [10]:
ssl._create_default_https_context = ssl._create_unverified_context

# Lectura de presupuestos desde dropbox
df_proyectado = pd.read_excel("https://www.dropbox.com/s/m96bbvnctgmaxrz/Proyecci%C3%B3n%20mensual%20%282023%29.xlsx?dl=1",sheet_name=mes_actual2 ,engine="openpyxl")
# OJO !! El link original tiene un 0 para ser visto desde la web y un 1 para descargarlo.....
df_proyectado.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4212000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.4-2 LECTURA "PROYECTADOS 2"

In [11]:
# Get the current date
import datetime

fecha_actual = datetime.datetime.now()

# Calculate the previous month
mes_anterior2 = (fecha_actual - timedelta(days=30)).month
print(mes_anterior2)

if mes_anterior2 == 1:
    mes_anterior2 = "ENERO"
elif mes_anterior2 == 2:
    mes_anterior2 = "FEBRERO"
elif mes_anterior2 == 3:
    mes_anterior2 = "MARZO"
elif mes_anterior2 == 4:
    mes_anterior2 = "ABRIL"
elif mes_anterior2 == 5:
    mes_anterior2 = "MAYO"
elif mes_anterior2 == 6:
    mes_anterior2 = "JUNIO"
elif mes_anterior2 == 7:
    mes_anterior2 = "JULIO"
elif mes_anterior2 == 8:
    mes_anterior2 = "AGOSTO"
elif mes_anterior2 == 9:
    mes_anterior2 = "SEPTIEMBRE"
elif mes_anterior2 == 10:
    mes_anterior2 = "OCTUBRE"
elif mes_anterior2 == 11:
    mes_anterior2 = "NOVIEMBRE"
else:
    mes_anterior2 = "DICIEMBRE"

print(mes_anterior2)

# Lectura de presupuestos desde dropbox
df_proyectado_1M = pd.read_excel("https://www.dropbox.com/s/m96bbvnctgmaxrz/Proyecci%C3%B3n%20mensual%20%282023%29.xlsx?dl=1",sheet_name=mes_anterior2 ,engine="openpyxl")
# OJO !! El link original tiene un 0 para ser visto desde la web y un 1 para descargarlo.....
df_proyectado_1M.head(10)

5
MAYO


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.5 LECTURA "LIBRO MAYOR AÑOS ANTERIORES"

In [574]:
# Conexión a google drive

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# Credenciales de GOOGLE API, en archivo .json
credentials = ServiceAccountCredentials.from_json_keyfile_name(
         'keen-extension-358919-9214486a06be.json', scope) # Your json file here

gc = gspread.authorize(credentials)

## SIEMPRE DAR ACCESO A ESTE USUARIO: test-321@keen-extension-358919.iam.gserviceaccount.com

In [575]:
spr = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Fx1nerTu10PZPOwknLcQuri_I6ARat5h7zuq4P3vypg/edit#gid=1097918290')
recipients = spr.sheet1.get_all_values()

# Se el string creado en un dataframe
df_libro_mayor_old = pd.DataFrame(recipients)

# se reasignan los headers en el dataframe.
new_header = df_libro_mayor_old.iloc[0] #grab the first row for the header
df_libro_mayor_old = df_libro_mayor_old[1:] #take the data less the header row
df_libro_mayor_old.columns = new_header #set the header row as the df header

# Se conservan solo las columnas de interés
#df_seguridad_v3 = df_seguridad_v3[["Nº Cédula","Nombres y apellidos","APRUEBA FASE DE CALIFICACIÓN ?","PUNTOS VIGENTES","APRUEBA FASE DE EVALUACIÓN ?","RESULTADO FINAL"]]

print("Las dimensiones del dataframe son :",df_libro_mayor_old.shape)
df_libro_mayor_old.sample(5)

Las dimensiones del dataframe son : (17368, 13)


,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario
9580,4101010-000,2022,3,3/14/22 0:00,CIN,245,"Teflon 3/4"" X 10M",P-1685,I15412322,44749,10450,0,
6122,4101010-000,2021,11,11/15/21 0:00,CIN,351,Camara Bala IR Varif 2.8-12mm HD-TVI 1MP Hikvi...,P-1656,S09221021,43702,0,18476,
14846,4101010-000,2022,11,11/21/22 0:00,CIN,627,Electrodo Soldadura 7018 3/32,P-1712,I15412456,48128,12538,0,
9465,4101010-000,2022,3,3/1/22 0:00,CIN,381,Citofono GT-1D Aiphone,P-1669,C04080111,44650,0,232650,
15664,4150002-000,2022,2,2/28/22 0:00,AJC,939,REMUNERACIONES DIRECTAS PROYECTO,P-1670,P-1670,11474055,70790,0,


In [576]:
df_libro_mayor_old.to_csv('archivos_respaldo/df_libro_mayor_old.csv', mode='a')

### 2.6 LECTURA "PLANILLA RESUMEN - PRESPUESTOS"

In [577]:
# Lectura de presupuestos desde dropbox
df_presupuesto = pd.read_excel("https://www.dropbox.com/s/miow7p9wl7t3jsr/PLANILLA%20RESUMEN.xlsb.xlsx?dl=1",sheet_name= "PROYECTOS" ,engine="openpyxl")
# OJO !! El link original tiene un 0 para ser visto desde la web y un 1 para descargarlo.....
df_presupuesto.head(5)

,PROYECTO,OBRA,RUT,CONSTRUCTORA,FECHA INICIO,FECHA TERMINO,ULTIMO EP PRESENTADO,AVANCE EN OBRA,MONTO EN $,$ POR FACTURAR,...,217465.036046312,217465.036046312.1,217465.036046312.2,217465.036046312.3,0,0.1,0.2,0.3,0.4,0.5
0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,8111745112.45,3283770104.37,...,1,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00,10.00
1,P-1632,Chorrillos Clorox,76.108.978-1,SuKsa S.A,2019-02-04 00:00:00,2019-05-31,NaN,0.99,138107706.03,915642.26,...,217465.04,217465.04,217465.04,217465.04,0.00,0.00,0.00,0.00,0.00,0.00
2,P-1638,Colon,78134800-7,Grevia,2019-08-13 00:00:00,2020-03-27,NaN,1.00,43206282.00,2177592.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P-1645,Los 3 Antonios,77.710.830-1,EDICASA,2019-08-31 00:00:00,NaT,NaN,0.98,18150000.00,208732.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P-1647,Bodega corrosivos TW,76.564.630-8,Inmobiliaria y Rentas Logisticas S.A,NaN,NaT,NaN,0.58,29482131.00,12437344.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [578]:
df_presupuesto.to_csv('archivos_respaldo/df_presupuesto.csv', mode='a')

### 2.7 LECTURA "TABLA- PRESPUESTOS"

In [579]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\Presupuesto Proyectos.txt'
response = requests.get(url)
query_result = response.text
soup = BeautifulSoup(response.content, 'xml')
rows = soup.find_all('Table')


data = []
for row in rows:
    values = [cell.text for cell in row.find_all()]
    data.append(values)

df_tabla_prespuesto = pd.DataFrame(data, columns=['CODIGO_PROYECTO','PARTIDA_PRESUPUESTARIA','PRODUCTO','UNIDAD','CANTIDAD','COSTO_UNITARIO','COSTO_TOTAL'])

# Exportación a excel
df_tabla_prespuesto.to_csv('df_tabla_prespuesto.csv', index = False)

df_tabla_prespuesto.head(5)

,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL
0,P-1602,1001,DETECCIÓN-Central de detección Hochiki FNP112...,,1,428490,428490
1,P-1602,1002,DETECCIÓN-Sensor Humo Hochiki ALN-C,,49,20866,1022434
2,P-1602,1003,DETECCIÓN-Sensor de temperatura Hochiki ATJ-EA,,4,23846,95384
3,P-1602,1004,DETECCIÓN-Base Sensores Hochiki HSB-NSA-6,,53,2608,138224
4,P-1602,1005,DETECCIÓN-Palanca Manual Hochiki DCP-AMS,,13,27200,353600


## 3.- TRATAMIENTO DE DATAFRAMES

### 3.1 - TRATAMIENTO 1 - LIBRO MAYOR

#### 3.1.1 - Exploración inicial de los datos

In [580]:
df_libro_mayor.shape

(7243, 12)

In [581]:
df_libro_mayor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7243 entries, 0 to 7242
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Cta_Contable     7243 non-null   object
 1   Año              7243 non-null   object
 2   Mes              7243 non-null   object
 3   Fecha            7218 non-null   object
 4   Tipo_Comp        7243 non-null   object
 5   Num_Comp         7243 non-null   object
 6   Glosa            7243 non-null   object
 7   Primer_Analisis  7240 non-null   object
 8   Referencia       7243 non-null   object
 9   Debito           7243 non-null   object
 10  Credito          7243 non-null   object
 11  Num_Referencia   6039 non-null   object
dtypes: object(12)
memory usage: 679.2+ KB


#### 3.1.2 - Estandarización campo "Debito" y Crédito

In [582]:
# Se crean nuevos campos para trabajar "Debito" y "Credito".

df_libro_mayor['Debito_2'] = pd.to_numeric(df_libro_mayor['Debito'], errors='coerce')
df_libro_mayor['Credito_2'] = pd.to_numeric(df_libro_mayor['Credito'], errors='coerce')

# Se reemplaza el valor de los NoNe y Nan 

df_libro_mayor['Credito_2'] = df_libro_mayor['Credito_2'].fillna(0)

df_libro_mayor.head(5)

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2
0,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,173,VENTAS DE PROYECTO,P-1677,P-1677,4081983,0,NaN,4081983,0
1,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,174,VENTAS DE PROYECTO,P-1677,P-1677,6464848,0,NaN,6464848,0
2,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,824,VENTAS DE PROYECTO,P-1700,P-1700,4673685,0,10604,4673685,0
3,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,825,VENTAS DE PROYECTO,P-1700,P-1700,1570055,0,10603,1570055,0
4,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,827,VENTAS DE PROYECTO,P-1704,P-1704,3613652,0,10602,3613652,0


#### 3.1.3 - Creación campo Codigo_proyecto

In [583]:
# Se crean una nueva columna para manejar el código del proyecto 

df_libro_mayor['Codigo_proyecto'] = df_libro_mayor['Primer_Analisis']

cols = df_libro_mayor.columns.tolist() # Obtiene el nombre de todas las columnas en una lista
cols.insert(0, cols.pop(cols.index('Codigo_proyecto'))) # Extrae la columna 'Codigo_proyecto', la inserta en la posición 0 y actualiza la lista
df_libro_mayor = df_libro_mayor.reindex(columns=cols) # Reindexa el dataframe con las columnas en el orden actualizado

# Transformación a mayúscula código del proyecto
df_libro_mayor['Codigo_proyecto'] = df_libro_mayor['Codigo_proyecto'].str.upper()

df_libro_mayor.head(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2
0,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,173,VENTAS DE PROYECTO,P-1677,P-1677,4081983,0,NaN,4081983,0
1,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,174,VENTAS DE PROYECTO,P-1677,P-1677,6464848,0,NaN,6464848,0
2,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,824,VENTAS DE PROYECTO,P-1700,P-1700,4673685,0,10604,4673685,0
3,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,825,VENTAS DE PROYECTO,P-1700,P-1700,1570055,0,10603,1570055,0
4,P-1704,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,827,VENTAS DE PROYECTO,P-1704,P-1704,3613652,0,10602,3613652,0


#### 3.1.4 - Creación de campo total

In [584]:
df_libro_mayor['TOTAL'] = df_libro_mayor['Credito_2'] - df_libro_mayor['Debito_2']
df_libro_mayor.head(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL
0,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,173,VENTAS DE PROYECTO,P-1677,P-1677,4081983,0,NaN,4081983,0,-4081983
1,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,174,VENTAS DE PROYECTO,P-1677,P-1677,6464848,0,NaN,6464848,0,-6464848
2,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,824,VENTAS DE PROYECTO,P-1700,P-1700,4673685,0,10604,4673685,0,-4673685
3,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,825,VENTAS DE PROYECTO,P-1700,P-1700,1570055,0,10603,1570055,0,-1570055
4,P-1704,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,827,VENTAS DE PROYECTO,P-1704,P-1704,3613652,0,10602,3613652,0,-3613652


#### 3.1.5 - Creación campo tipo_cuenta "basado en el campo cta_Contable"

In [585]:
# Función para categorizar la columna 'Cta_Contable'
def categorizar_cta_contable(cta_contable):
    if cta_contable =="3101030-000" or cta_contable == "3401002-000":
        return 'INGRESO'
    elif cta_contable == "4101010-000":
        return 'COSTO MATERIALES'
    elif cta_contable == "4150002-000":
        return 'COSTO MANO DE OBRA'
    elif cta_contable == "4150002-002":
        return 'COSTO SUPERVISIÓN'
    else:
        return 'OTRA'

# Crear la nueva columna 'Tipo_cuenta' aplicando la función
df_libro_mayor['Tipo_cuenta'] = df_libro_mayor['Cta_Contable'].apply(categorizar_cta_contable)
df_libro_mayor.sample(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta
5217,P-1723,4101010-000,2023,5,2023-05-08T00:00:00-03:00,CIN,210,DERIVADOR 5-1200MHZ RIT ALAMBRICA 4D 8DB,P-1723,C08712948,35688,0,50966,35688,0,-35688,COSTO MATERIALES
3202,P-1718,4101010-000,2023,3,2023-03-22T00:00:00-03:00,CIN,928,Set paños de limpieza,P-1718,I17440550,2353,0,49968,2353,0,-2353,COSTO MATERIALES
3408,P-1728,4101010-000,2023,3,2023-03-31T00:00:00-03:00,CIN,1590,Tee 90 mm HDPE Electrofusion,P-1728,O19501068,22509,0,50135,22509,0,-22509,COSTO MATERIALES
5403,P-1686,4101010-000,2023,5,2023-05-11T00:00:00-03:00,CIN,573,Derivador tap 8 sal 17dB 5-1200MHz alambrica,P-1686,C06111966,25596,0,51103,25596,0,-25596,COSTO MATERIALES
1332,P-1699,4101010-000,2023,1,2023-01-19T00:00:00-03:00,CIN,741,Cinta Permace 3/4,P-1699,I16430109,2132,0,48999,2132,0,-2132,COSTO MATERIALES


In [586]:
# EXPLORACIÓN DE DATOS

df_libro_mayor_piv = pd.pivot_table(df_libro_mayor, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_piv = pd.DataFrame(df_libro_mayor_piv.to_records())

#df_libro_mayor_piv.head(5)
df_libro_mayor_piv.sample(10)

,Tipo_cuenta,Año,Mes,Credito_2,Debito_2,TOTAL
9,COSTO MATERIALES,2023,6,3333888,53619159,-50285271
2,COSTO MANO DE OBRA,2023,3,0,96802066,-96802066
14,INGRESO,2023,1,449490833,191247845,258242988
23,OTRA,2023,4,3,26395039,-26395036
0,COSTO MANO DE OBRA,2023,1,0,72542885,-72542885
7,COSTO MATERIALES,2023,4,3247244,151372142,-148124898
16,INGRESO,2023,3,481925304,199901371,282023933
8,COSTO MATERIALES,2023,5,8882684,177855466,-168972782
5,COSTO MATERIALES,2023,2,3348963,83938903,-80589940
19,INGRESO,2023,6,33681393,0,33681393


#### 3.1.6 - Se eliminan números de proyectos que no deben ser parte del análisis

In [587]:
# Se crea una copia del dataframe original debido que se reduciran las dimensiones
df_libro_mayor2 = df_libro_mayor.copy()

In [588]:
# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor2["Codigo_proyecto"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor2.shape)

Proyectos disponibles antes de la reducción : 
['P-1677' 'P-1700' 'P-1704' 'P-1716' 'P-1709' 'P-1687' 'P-1690' 'P-1686'
 'P-1688' 'P-1682' 'P-1694' 'P-1695' 'P-1703' 'P-1680' 'P-1697' 'P-1714'
 'P-1696' 'P-1722' 'P-1692' 'P-1718' 'P-1721' 'P-1715' 'P-1645' 'P-1699'
 'P-1701' 'P-1675' 'P-1684' 'P-1727' 'P-1723' 'P-1725' 'P-1672' 'P-1713'
 'P-1706' 'P-1726' 'P-1707' 'P-1724' 'P-1712' 'P-1705' 'P-1670' 'P-1691'
 'P-1720' 'P-1719' 'P-1681' 'P-1698' 'P-1708' 'P-PROYECTOS' 'P-1669'
 'P-1667' 'P-1668' 'P-1641' 'P-1693' 'P-1676' 'P-1689' 'P-1664' 'P-1658'
 'P-1660' 'P-1644' 'P-1592' 'P-1648' 'P-1611' 'P-1679' 'P-1642' 'P-1653'
 'P-1673' 'P-1683' 'P-1661' 'P-1612' 'P-1728' 'P-SERVICIOS' 'P-1639'
 'P-1674' 'P1718' 'P-1620' 'P-1603' 'P-1626' 'P-1649' 'P-1685' 'P-1632'
 'P-1643' 'P-1666' 'P-1656' '15314868-6' 'P-1717' 'P-1663' '78540190-5'
 '5555555-9' 'P-OFICINA' 'P-ANALISIS PROYE' 'P-1557' 'P-1591' '18800250-1'
 '13056802-5' '16411600-K' nan]
Dimensiones del dataframe antes de la reducción : 
(7

##### 3.1.6.1 - Se Se crea un subdataframe para COSTOS SUPERVISIÓN (separando lo que tiene P-OFICINA y cta_contable 4150002-002)

In [589]:
# Se crea un dataframe separado para tratar los que poseen proyecto "P-OFICINA" ya que los costos de supervisión solo aplican al total
df_libro_mayor2_2 = df_libro_mayor2.copy()

# Filtrar el DataFrame
df_libro_mayor2_2 = df_libro_mayor2_2[(df_libro_mayor2_2['Codigo_proyecto'] == 'P-OFICINA') & (df_libro_mayor2_2['Cta_Contable'] == '4150002-002')]

# Mostrar el DataFrame filtrado
df_libro_mayor2_2

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta
6950,P-OFICINA,4150002-002,2023,1,2023-01-31T00:00:00-03:00,AJC,1606,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2153838,0,NaN,2153838,0,-2153838,COSTO SUPERVISIÓN
6951,P-OFICINA,4150002-002,2023,1,2023-01-31T00:00:00-03:00,AJC,1606,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN
6952,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,1833838,0,NaN,1833838,0,-1833838,COSTO SUPERVISIÓN
6953,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN
6954,P-OFICINA,4150002-002,2023,3,2023-03-31T00:00:00-03:00,AJC,1694,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2292504,0,NaN,2292504,0,-2292504,COSTO SUPERVISIÓN
6955,P-OFICINA,4150002-002,2023,3,2023-03-31T00:00:00-03:00,AJC,1694,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN
6956,P-OFICINA,4150002-002,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2533436,0,NaN,2533436,0,-2533436,COSTO SUPERVISIÓN
6957,P-OFICINA,4150002-002,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2562685,0,NaN,2562685,0,-2562685,COSTO SUPERVISIÓN


In [590]:
# Elimina los registros NaN en la columna Codigo_proyecto
df_libro_mayor2 = df_libro_mayor2.dropna(subset=['Codigo_proyecto'])

# Define una lista de valores a eliminar
valores_a_eliminar = ['P-PROYECTOS', 'P-SERVICIOS','P-OFICINA', 'P-ANALISIS PROYE']

# Crea una máscara booleana que identifica las filas que contienen los valores a eliminar
mask1 = df_libro_mayor2['Codigo_proyecto'].isin(valores_a_eliminar)

# Crea una máscara booleana que identifica las filas que comienzan con un número
mask2 = df_libro_mayor2['Codigo_proyecto'].str[0].str.isnumeric()

# Une las dos máscaras con el operador OR
mask = mask1 | mask2

# Crea un nuevo dataframe sin las filas que contienen los valores a eliminar
df_libro_mayor2 = df_libro_mayor2[~mask]


# Verifica si el valor 'P1718' o valore similares están presentes en la columna Codigo_proyecto
if 'P1718' in df_libro_mayor2['Codigo_proyecto'].values:
    # Reemplaza el valor 'P1718' por 'P-1718' en la columna Codigo_proyecto
    df_libro_mayor2['Codigo_proyecto'] = df_libro_mayor2['Codigo_proyecto'].replace('P1718', 'P-1718')


In [591]:
# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor2["Codigo_proyecto"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor2.shape)

Proyectos disponibles antes de la reducción : 
['P-1677' 'P-1700' 'P-1704' 'P-1716' 'P-1709' 'P-1687' 'P-1690' 'P-1686'
 'P-1688' 'P-1682' 'P-1694' 'P-1695' 'P-1703' 'P-1680' 'P-1697' 'P-1714'
 'P-1696' 'P-1722' 'P-1692' 'P-1718' 'P-1721' 'P-1715' 'P-1645' 'P-1699'
 'P-1701' 'P-1675' 'P-1684' 'P-1727' 'P-1723' 'P-1725' 'P-1672' 'P-1713'
 'P-1706' 'P-1726' 'P-1707' 'P-1724' 'P-1712' 'P-1705' 'P-1670' 'P-1691'
 'P-1720' 'P-1719' 'P-1681' 'P-1698' 'P-1708' 'P-1669' 'P-1667' 'P-1668'
 'P-1641' 'P-1693' 'P-1676' 'P-1689' 'P-1664' 'P-1658' 'P-1660' 'P-1644'
 'P-1592' 'P-1648' 'P-1611' 'P-1679' 'P-1642' 'P-1653' 'P-1673' 'P-1683'
 'P-1661' 'P-1612' 'P-1728' 'P-1639' 'P-1674' 'P-1620' 'P-1603' 'P-1626'
 'P-1649' 'P-1685' 'P-1632' 'P-1643' 'P-1666' 'P-1656' 'P-1717' 'P-1663'
 'P-1557' 'P-1591']
Dimensiones del dataframe antes de la reducción : 
(6871, 17)


#### 3.1.7 - Se crea un nuevo campo de fecha basado en la fecha original

In [592]:
df_libro_mayor2['Fecha2'] = pd.to_datetime(df_libro_mayor2['Año'].astype(str) + '-' + df_libro_mayor2['Mes'].astype(str) + '-1')
df_libro_mayor2.sample(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
3365,P-1691,4101010-000,2023,3,2023-03-28T00:00:00-03:00,CIN,1249,Cinta Aisladora 3M Temflex 1500,P-1691,I16430107,1576,0,50093,1576,0,-1576,COSTO MATERIALES,2023-03-01
1002,P-1716,4101010-000,2023,1,2023-01-09T00:00:00-03:00,CIN,182,Caja electrica Galvanizada A-01 100 x 65,P-1716,T14370774,17670,0,48814,17670,0,-17670,COSTO MATERIALES,2023-01-01
6055,P-1694,4101010-000,2023,6,2023-06-02T00:00:00-04:00,CIN,60,Vaselina Vegetal,P-1694,I17452714,9807,0,51489,9807,0,-9807,COSTO MATERIALES,2023-06-01
3523,P-1723,4101010-000,2023,4,2023-04-03T00:00:00-03:00,CIN,39,Caluga fijador amarra plastica Blanca,P-1723,I16430045,2400,0,50167,2400,0,-2400,COSTO MATERIALES,2023-04-01
4393,P-1699,4101010-000,2023,4,2023-04-20T00:00:00-03:00,CIN,878,"Tornillo Roscalata 8 x 1 1/2""",P-1699,I17440455,1700,0,50595,1700,0,-1700,COSTO MATERIALES,2023-04-01


##### 3.1.7.1 - Se crea un nuevo campo de fecha basado en la fecha original para  COSTOS SUPERVISIÓN

In [593]:
df_libro_mayor2_2['Fecha2'] = pd.to_datetime(df_libro_mayor2_2['Año'].astype(str) + '-' + df_libro_mayor2_2['Mes'].astype(str) + '-1')
df_libro_mayor2_2.sample(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
6950,P-OFICINA,4150002-002,2023,1,2023-01-31T00:00:00-03:00,AJC,1606,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2153838,0,NaN,2153838,0,-2153838,COSTO SUPERVISIÓN,2023-01-01
6953,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN,2023-02-01
6952,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,1833838,0,NaN,1833838,0,-1833838,COSTO SUPERVISIÓN,2023-02-01
6955,P-OFICINA,4150002-002,2023,3,2023-03-31T00:00:00-03:00,AJC,1694,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN,2023-03-01
6951,P-OFICINA,4150002-002,2023,1,2023-01-31T00:00:00-03:00,AJC,1606,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN,2023-01-01


#### 3.1.8 - Se realiza pivote final del dataframe

In [594]:
# Se exploran las columnas:
df_libro_mayor2.columns

Index(['Codigo_proyecto', 'Cta_Contable', 'Año', 'Mes', 'Fecha', 'Tipo_Comp',
       'Num_Comp', 'Glosa', 'Primer_Analisis', 'Referencia', 'Debito',
       'Credito', 'Num_Referencia', 'Debito_2', 'Credito_2', 'TOTAL',
       'Tipo_cuenta', 'Fecha2'],
      dtype='object')

In [595]:
# crear pivot table sin margenes

df_libro_mayor2_piv = pd.pivot_table(df_libro_mayor2, values=['Debito_2','Credito_2','TOTAL'], index=['Codigo_proyecto','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor2_piv = pd.DataFrame(df_libro_mayor2_piv.to_records())

df_libro_mayor2_piv.sample(5)

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
309,P-1705,COSTO MATERIALES,2023-04-01 00:00:00,2023,4,0,5634234,-5634234
247,P-1699,COSTO MATERIALES,2023-01-01 00:00:00,2023,1,44280,5755747,-5711467
389,P-1714,INGRESO,2023-01-01 00:00:00,2023,1,9806949,0,9806949
391,P-1714,INGRESO,2023-05-01 00:00:00,2023,5,2436155,0,2436155
266,P-1700,COSTO MATERIALES,2023-05-01 00:00:00,2023,5,13104,38040,-24936


In [596]:
# Estas son las dimensiones del dataframe pivoteado:
df_libro_mayor2_piv.shape

(503, 8)

##### 3.1.8.1 - Se realiza pivote final del dataframe paera COSTO SUPERVISIÓN

In [597]:
# crear pivot table sin margenes

df_libro_mayor2_2_piv = pd.pivot_table(df_libro_mayor2_2, values=['Debito_2','Credito_2','TOTAL'], index=['Codigo_proyecto','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor2_2_piv = pd.DataFrame(df_libro_mayor2_2_piv.to_records())

df_libro_mayor2_2_piv = df_libro_mayor2_2_piv[df_libro_mayor2_2_piv['Codigo_proyecto'] != 'TOTAL']

# Se reemplaza el valor P-OFICINA por el valor TOTAL con lo cual se utilizará para unirlo al dataframe que crea las filas para el cálculo de los totales
df_libro_mayor2_2_piv['Codigo_proyecto'] = df_libro_mayor2_2_piv['Codigo_proyecto'].replace('P-OFICINA', 'TOTAL')


df_libro_mayor2_2_piv

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
0,TOTAL,COSTO SUPERVISIÓN,2023-01-01 00:00:00,2023,1,0,4336206,-4336206
1,TOTAL,COSTO SUPERVISIÓN,2023-02-01 00:00:00,2023,2,0,4016206,-4016206
2,TOTAL,COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023,3,0,4474872,-4474872
3,TOTAL,COSTO SUPERVISIÓN,2023-04-01 00:00:00,2023,4,0,5096121,-5096121


#### 3.1.9 - Se crea subdataframe para proyecto que considera el total con nombre "TOTAL", que se detalla por tipo_cuenta

In [598]:
df_libro_mayor_piv2 = pd.pivot_table(df_libro_mayor2_piv, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0)
df_libro_mayor_piv2 = pd.DataFrame(df_libro_mayor_piv2.to_records())

# Agregamos la columna Codigo_proyecto con valor "TOTAL" en la primera columna
df_libro_mayor_piv2.insert(loc=0, column='Codigo_proyecto', value='TOTAL')

# Eliminamos los registros con Tipo_cuenta igual a "TOTAL"
df_libro_mayor_piv2 = df_libro_mayor_piv2[df_libro_mayor_piv2['Tipo_cuenta'] != 'TOTAL']
df_libro_mayor_piv2 = df_libro_mayor_piv2[df_libro_mayor_piv2['Tipo_cuenta'] != '']

df_libro_mayor_piv2

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
1,TOTAL,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023,1,0,72482994,-72482994
2,TOTAL,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023,2,0,82645544,-82645544
3,TOTAL,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023,3,0,96760244,-96760244
4,TOTAL,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023,4,0,111885468,-111885468
5,TOTAL,COSTO MATERIALES,2023-01-01 00:00:00,2023,1,2565889,125972929,-123407040
6,TOTAL,COSTO MATERIALES,2023-02-01 00:00:00,2023,2,3348963,83658197,-80309234
7,TOTAL,COSTO MATERIALES,2023-03-01 00:00:00,2023,3,2764412,88079320,-85314908
8,TOTAL,COSTO MATERIALES,2023-04-01 00:00:00,2023,4,3247244,150098697,-146851453
9,TOTAL,COSTO MATERIALES,2023-05-01 00:00:00,2023,5,8533707,176803071,-168269364
10,TOTAL,COSTO MATERIALES,2023-06-01 00:00:00,2023,6,3333888,53436096,-50102208


##### 3.1.9.1 - Se anexa el dataframe creado con los totales con el dataframe creado a partir del punto 3.1.6.1 que contiene los costos de supervisión

In [599]:
df_libro_mayor_piv2 = pd.concat([df_libro_mayor_piv2, df_libro_mayor2_2_piv], ignore_index=True)
df_libro_mayor_piv2

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
0,TOTAL,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023,1,0,72482994,-72482994
1,TOTAL,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023,2,0,82645544,-82645544
2,TOTAL,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023,3,0,96760244,-96760244
3,TOTAL,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023,4,0,111885468,-111885468
4,TOTAL,COSTO MATERIALES,2023-01-01 00:00:00,2023,1,2565889,125972929,-123407040
5,TOTAL,COSTO MATERIALES,2023-02-01 00:00:00,2023,2,3348963,83658197,-80309234
6,TOTAL,COSTO MATERIALES,2023-03-01 00:00:00,2023,3,2764412,88079320,-85314908
7,TOTAL,COSTO MATERIALES,2023-04-01 00:00:00,2023,4,3247244,150098697,-146851453
8,TOTAL,COSTO MATERIALES,2023-05-01 00:00:00,2023,5,8533707,176803071,-168269364
9,TOTAL,COSTO MATERIALES,2023-06-01 00:00:00,2023,6,3333888,53436096,-50102208


#### 3.1.10 - Se concatenan los 2 dataframes resultantes

In [600]:
print("=======================================================================")
print("Dimensiones del dataframe antes de la concatenación : ")
print(df_libro_mayor2_piv.shape)

Dimensiones del dataframe antes de la concatenación : 
(503, 8)


In [601]:
df_libro_mayor3 = pd.concat([df_libro_mayor2_piv, df_libro_mayor_piv2])
df_libro_mayor3.sort_values(by=['Codigo_proyecto']).sample(5)

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
410,P-1716,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023,4,0,895428,-895428
109,P-1684,COSTO MATERIALES,2023-03-01 00:00:00,2023,3,3200,717239,-714039
468,P-1724,INGRESO,2023-03-01 00:00:00,2023,3,15294733,0,15294733
314,P-1705,INGRESO,2023-05-01 00:00:00,2023,5,20190289,0,20190289
42,P-1667,COSTO MATERIALES,2023-05-01 00:00:00,2023,5,0,90852,-90852


In [602]:
print("=======================================================================")
print("Dimensiones del dataframe después de la concatenación : ")
print(df_libro_mayor3.shape)

Dimensiones del dataframe después de la concatenación : 
(523, 8)


#### 3.1.11 - Se crean las filas que permiten tener siempre todas las opciones de Tipo_Cuenta por cada Codigo_proyecto

In [603]:
combinaciones = []
for proyecto in df_libro_mayor3['Codigo_proyecto'].unique():
    for fecha in df_libro_mayor3['Fecha2'].unique():
        mes = pd.to_datetime(fecha).month
        año = pd.to_datetime(fecha).year
        for cuenta in ['INGRESO', 'COSTO MATERIALES', 'COSTO MANO DE OBRA', 'COSTO SUPERVISIÓN']:
            combinaciones.append((proyecto, fecha, mes, año, cuenta))

# crear un nuevo DataFrame con las combinaciones obtenidas
combinaciones_df = pd.DataFrame(combinaciones, columns=['Codigo_proyecto', 'Fecha2', 'Mes', 'Año', 'Tipo_cuenta'])

# agregar las demás columnas con valores nulos
combinaciones_df['Credito_2'] = 0
combinaciones_df['Debito_2'] = 0
combinaciones_df['TOTAL'] = 0

# Se eliminan filas con fecha en blanco.
combinaciones_df = combinaciones_df.dropna(subset=['Mes'])

combinaciones_df.sample(10)

,Codigo_proyecto,Fecha2,Mes,Año,Tipo_cuenta,Credito_2,Debito_2,TOTAL
766,P-1669,2023-02-01 00:00:00,2.00,2023.00,COSTO MANO DE OBRA,0,0,0
42,P-1591,2023-03-01 00:00:00,3.00,2023.00,COSTO MANO DE OBRA,0,0,0
1741,P-1707,2023-01-01 00:00:00,1.00,2023.00,COSTO MATERIALES,0,0,0
2300,TOTAL,2023-01-01 00:00:00,1.00,2023.00,INGRESO,0,0,0
1614,P-1701,2023-05-01 00:00:00,5.00,2023.00,COSTO MANO DE OBRA,0,0,0
1801,P-1709,2023-02-01 00:00:00,2.00,2023.00,COSTO MATERIALES,0,0,0
1914,P-1715,2023-02-01 00:00:00,2.00,2023.00,COSTO MANO DE OBRA,0,0,0
171,P-1620,2023-04-01 00:00:00,4.00,2023.00,COSTO SUPERVISIÓN,0,0,0
512,P-1656,2023-02-01 00:00:00,2.00,2023.00,INGRESO,0,0,0
1866,P-1713,2023-05-01 00:00:00,5.00,2023.00,COSTO MANO DE OBRA,0,0,0


#### 3.1.12 - Se concatenan los 2 dataframes resultantes

In [604]:
print("=======================================================================")
print("Dimensiones del dataframe antes de la concatenación : ")
print(df_libro_mayor3.shape)

Dimensiones del dataframe antes de la concatenación : 
(523, 8)


In [605]:
df_libro_mayor4 = pd.concat([df_libro_mayor3, combinaciones_df])
df_libro_mayor4.sort_values(by=['Codigo_proyecto']).sample(5)

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
1997,P-1718,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,0,0,0
1383,P-1693,COSTO SUPERVISIÓN,2023-02-01 00:00:00,2023.00,2.00,0,0,0
522,P-1656,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023.00,5.00,0,0,0
1637,P-1703,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,0,0,0
954,P-1677,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,0,0


In [606]:
print("=======================================================================")
print("Dimensiones del dataframe después de la concatenación : ")
print(df_libro_mayor4.shape)

Dimensiones del dataframe después de la concatenación : 
(2515, 8)


#### 3.1.13 - Estandarización de campos 

In [607]:
# Se estandarizan los campos con sus nombres y formatos, para un mayor entendimiento.

df_libro_mayor4 = df_libro_mayor4.rename(columns={
    'Codigo_proyecto': 'CODIGO_PROYECTO',
    'Tipo_cuenta': 'TIPO_CUENTA',
    'Fecha2': 'FECHA',
    'Año': 'AÑO',
    'Mes': 'MES',
    'Credito_2': 'CREDITO',
    'Debito_2': 'DEBITO'
})

df_libro_mayor4['AÑO'] = pd.to_numeric(df_libro_mayor4['AÑO'], downcast='integer')
df_libro_mayor4['MES'] = pd.to_numeric(df_libro_mayor4['MES'], downcast='integer')

df_libro_mayor4.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL
1726,P-1706,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023.00,5.00,0,0,0
2222,P-1726,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,0,0
294,P-1704,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,14242535,-14242535
680,P-1666,INGRESO,2023-02-01 00:00:00,2023.00,2.00,0,0,0
384,P-1714,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,0,2172835,-2172835
404,P-1715,INGRESO,2023-03-01 00:00:00,2023.00,3.00,12538094,0,12538094
2087,P-1721,COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023.00,3.00,0,0,0
2316,TOTAL,INGRESO,2023-06-01 00:00:00,2023.00,6.00,0,0,0
1023,P-1680,COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023.00,3.00,0,0,0
1226,P-1687,COSTO MANO DE OBRA,2023-06-01 00:00:00,2023.00,6.00,0,0,0


In [608]:
#df_libro_mayor4[df_libro_mayor4["CODIGO_PROYECTO"]=="P-1706"]
df_libro_mayor4[df_libro_mayor4["CODIGO_PROYECTO"]=="TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL
502,TOTAL,,,NaN,NaN,2201473074,1665348788,536124286
0,TOTAL,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,72482994,-72482994
1,TOTAL,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,82645544,-82645544
2,TOTAL,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,96760244,-96760244
3,TOTAL,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,111885468,-111885468
4,TOTAL,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,2565889,125972929,-123407040
5,TOTAL,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,3348963,83658197,-80309234
6,TOTAL,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,2764412,88079320,-85314908
7,TOTAL,COSTO MATERIALES,2023-04-01 00:00:00,2023.00,4.00,3247244,150098697,-146851453
8,TOTAL,COSTO MATERIALES,2023-05-01 00:00:00,2023.00,5.00,8533707,176803071,-168269364


#### 3.1.14 Creación de columna MARGEN

In [609]:
df_libro_mayor4_2 = df_libro_mayor4.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_libro_mayor4_2.groupby(['CODIGO_PROYECTO','MES']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

    
df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(5)

,CODIGO_PROYECTO,MES,MARGEN
130,P-1661,5.00,0.00
96,P-1649,1.00,0.00
54,P-1639,1.00,0.00
262,P-1687,5.00,0.00
245,P-1684,6.00,0.00


In [610]:
print("=======================================================================")
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor4_2  : ", df_libro_mayor4.shape)
print("=======================================================================")
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_margen          : ", df_margen.shape)
print(df_margen.shape)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor4_2  :  (2515, 8)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_margen          :  (498, 3)
(498, 3)


In [611]:
# CONCATENACIÓN DE DATAFRAMES

df_libro_mayor5 = pd.merge(df_libro_mayor4, df_margen, on=['CODIGO_PROYECTO', 'MES'], how='left')

# Se estandariza el valor TOTAL a 1.TOTAL para que pueda ser seleccionado desde los filtros del Dashboard.
df_libro_mayor5['CODIGO_PROYECTO'] = df_libro_mayor5['CODIGO_PROYECTO'].apply(lambda x: '1. TOTAL' if x == 'TOTAL' else x)

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5  : ", df_libro_mayor5.shape)
print(df_libro_mayor5.shape)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5  :  (2515, 9)
(2515, 9)


In [612]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5[df_libro_mayor5["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
502,1. TOTAL,,,NaN,NaN,2201473074,1665348788,536124286,NaN
503,1. TOTAL,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,72482994,-72482994,0.00
504,1. TOTAL,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,82645544,-82645544,0.00
505,1. TOTAL,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,96760244,-96760244,0.00
506,1. TOTAL,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,111885468,-111885468,0.00
507,1. TOTAL,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,2565889,125972929,-123407040,0.00
508,1. TOTAL,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,3348963,83658197,-80309234,0.00
509,1. TOTAL,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,2764412,88079320,-85314908,0.00
510,1. TOTAL,COSTO MATERIALES,2023-04-01 00:00:00,2023.00,4.00,3247244,150098697,-146851453,0.00
511,1. TOTAL,COSTO MATERIALES,2023-05-01 00:00:00,2023.00,5.00,8533707,176803071,-168269364,0.00


In [613]:
# INSPECCIÓN VISUAL 2
df_libro_mayor5[df_libro_mayor5["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
291,P-1704,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,7375493,-7375493,0.00
292,P-1704,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,10751091,-10751091,0.00
293,P-1704,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,12332472,-12332472,0.00
294,P-1704,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,14242535,-14242535,0.00
295,P-1704,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,0,20073319,-20073319,0.00
296,P-1704,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,139976,3388760,-3248784,0.00
297,P-1704,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,0,14768626,-14768626,0.00
298,P-1704,COSTO MATERIALES,2023-04-01 00:00:00,2023.00,4.00,0,20031042,-20031042,0.00
299,P-1704,COSTO MATERIALES,2023-05-01 00:00:00,2023.00,5.00,0,13967551,-13967551,0.00
300,P-1704,COSTO MATERIALES,2023-06-01 00:00:00,2023.00,6.00,0,3973960,-3973960,0.00


In [614]:
df_libro_mayor5["CODIGO_PROYECTO"].unique()

array(['P-1557', 'P-1591', 'P-1592', 'P-1603', 'P-1611', 'P-1612',
       'P-1620', 'P-1626', 'P-1632', 'P-1639', 'P-1641', 'P-1642',
       'P-1643', 'P-1644', 'P-1645', 'P-1648', 'P-1649', 'P-1653',
       'P-1656', 'P-1658', 'P-1660', 'P-1661', 'P-1663', 'P-1664',
       'P-1666', 'P-1667', 'P-1668', 'P-1669', 'P-1670', 'P-1672',
       'P-1673', 'P-1674', 'P-1675', 'P-1676', 'P-1677', 'P-1679',
       'P-1680', 'P-1681', 'P-1682', 'P-1683', 'P-1684', 'P-1685',
       'P-1686', 'P-1687', 'P-1688', 'P-1689', 'P-1690', 'P-1691',
       'P-1692', 'P-1693', 'P-1694', 'P-1695', 'P-1696', 'P-1697',
       'P-1698', 'P-1699', 'P-1700', 'P-1701', 'P-1703', 'P-1704',
       'P-1705', 'P-1706', 'P-1707', 'P-1708', 'P-1709', 'P-1712',
       'P-1713', 'P-1714', 'P-1715', 'P-1716', 'P-1717', 'P-1718',
       'P-1719', 'P-1720', 'P-1721', 'P-1722', 'P-1723', 'P-1724',
       'P-1725', 'P-1726', 'P-1727', 'P-1728', '1. TOTAL'], dtype=object)

In [615]:
df_libro_mayor5.to_csv('archivos_respaldo/df_libro_mayor5.csv', index = False)

### 3.1 - TRATAMIENTO 1 - LIBRO MAYOR SUBSETS

#### 3.1.16 - CREACIÓN DF **"YTD AÑO EN CURSO HASTA MES CERRADO"**

In [616]:
fecha_actual = datetime.date.today()
fecha_actual

datetime.date(2023, 6, 6)

In [617]:
# Se crea una copia del dataframe 

df_libro_mayor_ytd = df_libro_mayor5.copy()
df_libro_mayor_ytd.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1832,P-1698,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,0,0,0,0.00
1008,P-1660,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,0,0,0,0.00
794,P-1642,COSTO SUPERVISIÓN,2023-01-01 00:00:00,2023.00,1.00,0,0,0,0.00
1189,P-1669,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023.00,5.00,0,0,0,0.00
1813,P-1697,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023.00,5.00,0,0,0,0.00


In [618]:
# Se crea un filtro para obtener todos aquellos que tengan valores hasta el MES anterior al mes en curso

# Obtener el mes actual
mes_actual = datetime.date.today().month
print(mes_actual)

# Filtrar los registros del dataframe que corresponden al mes actual y a todos los meses anteriores
df_libro_mayor_ytd["FECHA"] = pd.to_datetime(df_libro_mayor_ytd["FECHA"])
df_libro_mayor_ytd_closed = df_libro_mayor_ytd[df_libro_mayor_ytd["FECHA"].dt.month < mes_actual]

df_libro_mayor_ytd_closed.sample(10)

6


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1462,P-1683,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00
1022,P-1660,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00
1372,P-1679,COSTO MATERIALES,2023-02-01,2023.00,2.00,0,0,0,0.00
2384,P-1724,COSTO MATERIALES,2023-03-01,2023.00,3.00,0,0,0,0.00
1885,P-1700,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00
521,1. TOTAL,COSTO SUPERVISIÓN,2023-03-01,2023.00,3.00,0,4474872,-4474872,0.00
1202,P-1670,COSTO SUPERVISIÓN,2023-01-01,2023.00,1.00,0,0,0,0.00
446,P-1721,COSTO MATERIALES,2023-04-01,2023.00,4.00,95208,2062191,-1966983,0.00
1332,P-1676,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00
2410,P-1725,COSTO SUPERVISIÓN,2023-03-01,2023.00,3.00,0,0,0,0.00


In [619]:
# crear pivot table sin margenes

df_libro_mayor_ytd_closed_piv = pd.pivot_table(df_libro_mayor_ytd_closed, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)
#df_libro_mayor4_piv = pd.pivot_table(df_libro_mayor4, values=['Debito_2','Credito_2','TOTAL'], index=['Codigo_proyecto','Tipo_cuenta'], columns = ['Fecha2'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')

df_libro_mayor_ytd_closed_piv = pd.DataFrame(df_libro_mayor_ytd_closed_piv.to_records())

df_libro_mayor_ytd_closed_piv.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
10,P-1591,COSTO SUPERVISIÓN,0
279,P-1715,INGRESO,40610937
173,P-1686,COSTO MATERIALES,-12682908
31,P-1620,INGRESO,0
167,P-1684,INGRESO,10044427
2,1. TOTAL,COSTO SUPERVISIÓN,-17923405
243,P-1704,INGRESO,220776196
294,P-1719,COSTO SUPERVISIÓN,0
43,P-1639,INGRESO,0
76,P-1656,COSTO MANO DE OBRA,0


#### 3.1.16-1 - CREACIÓN DF **"YTD AÑO EN CURSO HASTA MES CERRADO 2"**
Este dataframe es para considerar como mes cerrado 2 meses atrás, de esta forma se puede evaluar el mes cerrado.

In [620]:
# Obtener el mes actual
mes_actual = datetime.date.today().month
print(mes_actual)

# Calcular el mes que es dos meses antes del actual
mes_filtrado = mes_actual - 2 if mes_actual > 2 else (12 + mes_actual - 2) 

# Filtrar los registros del dataframe que corresponden al mes filtrado y a todos los meses anteriores
df_libro_mayor_ytd["FECHA"] = pd.to_datetime(df_libro_mayor_ytd["FECHA"])
df_libro_mayor_ytd_closed_2M = df_libro_mayor_ytd[df_libro_mayor_ytd["FECHA"].dt.month <= mes_filtrado]

df_libro_mayor_ytd_closed_2M.sample(10)

6


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
2426,P-1726,COSTO SUPERVISIÓN,2023-01-01,2023.00,1.00,0,0,0,0.00
1925,P-1703,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,0,0,0.00
2471,P-1728,INGRESO,2023-01-01,2023.00,1.00,0,0,0,0.00
554,P-1591,COSTO SUPERVISIÓN,2023-01-01,2023.00,1.00,0,0,0,0.00
144,P-1688,COSTO MATERIALES,2023-03-01,2023.00,3.00,620196,4747994,-4127798,0.00
603,P-1603,INGRESO,2023-02-01,2023.00,2.00,0,0,0,0.00
820,P-1643,COSTO MATERIALES,2023-02-01,2023.00,2.00,0,0,0,0.00
206,P-1695,COSTO MATERIALES,2023-03-01,2023.00,3.00,0,3488915,-3488915,0.00
188,P-1694,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,2619666,-2619666,0.00
935,P-1653,INGRESO,2023-01-01,2023.00,1.00,0,0,0,0.00


In [621]:
# crear pivot table sin margenes

df_libro_mayor_ytd_closed_2M = pd.pivot_table(df_libro_mayor_ytd_closed_2M, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)

df_libro_mayor_ytd_closed_2M = pd.DataFrame(df_libro_mayor_ytd_closed_2M.to_records())

df_libro_mayor_ytd_closed_2M.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
221,P-1698,COSTO MATERIALES,924
148,P-1680,COSTO MANO DE OBRA,-22758101
118,P-1670,COSTO SUPERVISIÓN,0
49,P-1642,COSTO MATERIALES,8880
282,P-1716,COSTO SUPERVISIÓN,0


#### 3.1.17 - CREACIÓN DF **LIBRO MAYOR PARA MES EN CURSO**

In [622]:
# Se crea un nuevo dataframe basado en df_libro_mayor5, la palabra "mec" significa mes en curso.
 
df_libro_mayor5_mec = df_libro_mayor5.copy()
df_libro_mayor5_mec.shape
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec  : ", df_libro_mayor5_mec.shape)
df_libro_mayor5_mec.sample(3)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec  :  (2515, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
337,P-1708,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,550014,-550014,0.00
1048,P-1661,COSTO MATERIALES,2023-06-01 00:00:00,2023.00,6.00,0,0,0,0.00
977,P-1656,COSTO MANO DE OBRA,2023-06-01 00:00:00,2023.00,6.00,0,0,0,0.00


In [623]:
# Se crea una nueva columna para categorizar el origen de los datos.

df_libro_mayor5_mec["ORIGEN"]="LIBRO MAYOR"
df_libro_mayor5_mec.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
2489,P-1728,COSTO MANO DE OBRA,2023-06-01 00:00:00,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
442,P-1721,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,2706033,-2706033,0.00,LIBRO MAYOR
850,P-1644,COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023.00,3.00,0,0,0,0.00,LIBRO MAYOR
2462,P-1727,COSTO SUPERVISIÓN,2023-05-01 00:00:00,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
718,P-1632,COSTO SUPERVISIÓN,2023-04-01 00:00:00,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR


In [624]:
# Se seleccionan los meses que son utiles para crear la vista de mes en curso.
# Para el real se necesita: INGRESO, COSTO MATERIALES, Y SUPERVISIÓN (ÉSTE ÚLTIMO DEL MES ANTERIOR)
# Para el proyectado se necesita: SUPERVISIÓN (DEL MES ANTERIOR)

In [625]:
# Filtro para TIPO_CUENTA y FECHA
from datetime import datetime, timedelta


df_libro_mayor5_mec['FECHA'] = pd.to_datetime(df_libro_mayor5_mec['FECHA'])

filtro_ingreso_materiales = ((df_libro_mayor5_mec['TIPO_CUENTA'] == 'INGRESO') | 
                             (df_libro_mayor5_mec['TIPO_CUENTA'] == 'COSTO MATERIALES')) & \
                            (df_libro_mayor5_mec['FECHA'].dt.month == datetime.now().month)

# Filtro para SUPERVISIÓN del mes anterior
mes_anterior = (datetime.now() - timedelta(days=32)).month
filtro_supervision_mes_anterior = (df_libro_mayor5_mec['TIPO_CUENTA'] == 'COSTO SUPERVISIÓN') & \
                                  (df_libro_mayor5_mec['FECHA'].dt.month == mes_anterior)


# Filtro para COSTO_MANO_OBRA del mes anterior
mes_anterior = (datetime.now() - timedelta(days=32)).month
filtro_mo_mes_anterior = (df_libro_mayor5_mec['TIPO_CUENTA'] == 'COSTO MANO DE OBRA') & \
                                  (df_libro_mayor5_mec['FECHA'].dt.month == mes_anterior)

# Combinar los filtros en un filtro general
filtro_general = filtro_ingreso_materiales | filtro_supervision_mes_anterior  | filtro_mo_mes_anterior

# Crear el sub-dataframe filtrado
df_libro_mayor5_mec2 = df_libro_mayor5_mec[filtro_general]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec2  : ", df_libro_mayor5_mec2.shape)

df_libro_mayor5_mec2.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec2  :  (362, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
1382,P-1679,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1503,P-1684,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
927,P-1649,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
2464,P-1727,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
1119,P-1666,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
1191,P-1669,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
2414,P-1725,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
616,P-1603,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
1694,P-1692,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2101,P-1712,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR


In [626]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5_mec2[df_libro_mayor5_mec2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
300,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,3973960,-3973960,0.00,LIBRO MAYOR
1957,P-1704,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1958,P-1704,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1959,P-1704,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
1960,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR


In [627]:
# INSPECCIÓN VISUAL 2
df_libro_mayor5_mec2[df_libro_mayor5_mec2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
512,1. TOTAL,COSTO MATERIALES,2023-06-01,2023.00,6.00,3333888,53436096,-50102208,0.00,LIBRO MAYOR
518,1. TOTAL,INGRESO,2023-06-01,2023.00,6.00,33681393,0,33681393,0.00,LIBRO MAYOR
2509,1. TOTAL,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2510,1. TOTAL,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2511,1. TOTAL,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
2512,1. TOTAL,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR


#### 3.1.17-1 - CREACIÓN DF **LIBRO MAYOR PARA MES EN CURSO 2**
Este dataframe es para considerar como mes en curso de 2 meses atrás, de esta forma se puede evaluar el mes cerrado.

In [628]:
# Se crea un nuevo dataframe basado en df_libro_mayor5, la palabra "mec" significa mes en curso.

df_libro_mayor5_mec_2M = df_libro_mayor5.copy()
df_libro_mayor5_mec_2M.shape
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  : ", df_libro_mayor5_mec_2M.shape)
df_libro_mayor5_mec_2M.sample(3)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  :  (2515, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
249,P-1699,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,0,3475458,-3475458,0.00
208,P-1695,COSTO MATERIALES,2023-05-01 00:00:00,2023.00,5.00,981954,9619052,-8637098,0.00
918,P-1649,COSTO SUPERVISIÓN,2023-02-01 00:00:00,2023.00,2.00,0,0,0,0.00


In [629]:
# Filtro para TIPO_CUENTA y FECHA
from datetime import datetime, timedelta

df_libro_mayor5_mec_2M['FECHA'] = pd.to_datetime(df_libro_mayor5_mec_2M['FECHA'])

# Filtro para INGRESO Y COSTO DE MATERIALES
mes_anterior = (datetime.now() - timedelta(days=30)).month

filtro_ingreso_materiales = ((df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'INGRESO') | 
                             (df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'COSTO MATERIALES')) & \
                            (df_libro_mayor5_mec_2M['FECHA'].dt.month == mes_anterior)

# Filtro para SUPERVISIÓN del mes anterior
mes_anterior2 = (datetime.now() - timedelta(days=60)).month

filtro_supervision_mes_anterior = (df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'COSTO SUPERVISIÓN') & \
                                  (df_libro_mayor5_mec_2M['FECHA'].dt.month == mes_anterior2)


# Filtro para COSTO_MANO_OBRA del mes anterior
mes_anterior2 = (datetime.now() - timedelta(days=60)).month

filtro_mo_mes_anterior = (df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'COSTO MANO DE OBRA') & \
                                  (df_libro_mayor5_mec_2M['FECHA'].dt.month == mes_anterior2)

# Combinar los filtros en un filtro general
filtro_general = filtro_ingreso_materiales | filtro_supervision_mes_anterior  | filtro_mo_mes_anterior

# Crear el sub-dataframe filtrado
df_libro_mayor5_mec_2M = df_libro_mayor5_mec_2M[filtro_general]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  : ", df_libro_mayor5_mec_2M.shape)

df_libro_mayor5_mec_2M.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  :  (446, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1355,P-1677,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00
1317,P-1676,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00
438,P-1720,INGRESO,2023-05-01,2023.00,5.00,21381860,21381860,0,0.00
118,P-1686,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,4634424,-4634424,0.00
239,P-1697,INGRESO,2023-05-01,2023.00,5.00,3945494,0,3945494,0.00


In [630]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5_mec_2M[df_libro_mayor5_mec_2M["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
294,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,14242535,-14242535,0.00
299,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,13967551,-13967551,0.00
305,P-1704,INGRESO,2023-05-01,2023.00,5.00,132309997,53250000,79059997,0.00
1941,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00
1942,P-1704,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00
1955,P-1704,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00
1956,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00


In [631]:
# INSPECCIÓN VISUAL 2
df_libro_mayor5_mec_2M[df_libro_mayor5_mec_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
506,1. TOTAL,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,111885468,-111885468,0.00
511,1. TOTAL,COSTO MATERIALES,2023-05-01,2023.00,5.00,8533707,176803071,-168269364,0.00
517,1. TOTAL,INGRESO,2023-05-01,2023.00,5.00,572605764,170698310,401907454,0.00
522,1. TOTAL,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,5096121,-5096121,0.00
2493,1. TOTAL,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00
2494,1. TOTAL,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00
2507,1. TOTAL,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00
2508,1. TOTAL,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00


#### 3.1.18 - CREACIÓN DF **LIBRO MAYOR PARA YTD MES PROYECTADO (Incluyendo mes actual)**

In [632]:
# Se crea una copia del dataframe 

df_libro_mayor_ytd2 = df_libro_mayor5.copy()
df_libro_mayor_ytd2.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
854,P-1644,COSTO SUPERVISIÓN,2023-05-01 00:00:00,2023.00,5.00,0,0,0,0.00
1680,P-1692,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,0,0,0,0.00
2487,P-1728,INGRESO,2023-06-01 00:00:00,2023.00,6.00,0,0,0,0.00
911,P-1649,INGRESO,2023-01-01 00:00:00,2023.00,1.00,0,0,0,0.00
643,P-1612,INGRESO,2023-04-01 00:00:00,2023.00,4.00,0,0,0,0.00
588,P-1592,COSTO MATERIALES,2023-05-01 00:00:00,2023.00,5.00,0,0,0,0.00
1845,P-1699,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,0,0,0.00
1865,P-1699,COSTO MANO DE OBRA,2023-06-01 00:00:00,2023.00,6.00,0,0,0,0.00
211,P-1695,INGRESO,2023-03-01 00:00:00,2023.00,3.00,3163875,0,3163875,0.00
2370,P-1723,COSTO SUPERVISIÓN,2023-06-01 00:00:00,2023.00,6.00,0,0,0,0.00


In [633]:
# crear pivot table sin margenes

df_libro_mayor_ytd2_closed_piv = pd.pivot_table(df_libro_mayor_ytd2, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)

df_libro_mayor_ytd2_closed_piv = pd.DataFrame(df_libro_mayor_ytd2_closed_piv.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_closed_piv  : ", df_libro_mayor_ytd2_closed_piv.shape)

df_libro_mayor_ytd2_closed_piv.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_closed_piv  :  (333, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
164,P-1683,INGRESO,0
45,P-1641,COSTO MANO DE OBRA,0
163,P-1683,COSTO SUPERVISIÓN,0
138,P-1676,COSTO MATERIALES,-368890
180,P-1687,INGRESO,19576547
134,P-1675,COSTO MATERIALES,0
319,P-1725,COSTO SUPERVISIÓN,0
2,1. TOTAL,COSTO MATERIALES,-654254207
252,P-1706,INGRESO,36302208
236,P-1701,INGRESO,1393986


#### 3.1.18-2 - CREACIÓN DF **LIBRO MAYOR PARA YTD MES PROYECTADO (Incluyendo mes actual) 2**

In [634]:
# Obtener el mes actual
mes_actual = datetime.now().month

# Filtrar los registros hasta el mes anterior al mes actual
df_libro_mayor_ytd2_1M = df_libro_mayor_ytd2[df_libro_mayor_ytd2['MES'] < mes_actual]

# Mostrar el nuevo DataFrame
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_1M  : ", df_libro_mayor_ytd2_1M.shape)
df_libro_mayor_ytd2_1M.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_1M  :  (2152, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1855,P-1699,INGRESO,2023-03-01 00:00:00,2023.00,3.00,0,0,0,0.00
1853,P-1699,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,0,0,0.00
2210,P-1717,COSTO SUPERVISIÓN,2023-01-01 00:00:00,2023.00,1.00,0,0,0,0.00
214,P-1696,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,4663709,-4663709,0.00
522,1. TOTAL,COSTO SUPERVISIÓN,2023-04-01 00:00:00,2023.00,4.00,0,5096121,-5096121,0.00


In [635]:
# crear pivot table sin margenes

df_libro_mayor_ytd2_closed_piv_1M = pd.pivot_table(df_libro_mayor_ytd2_1M, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)

df_libro_mayor_ytd2_closed_piv_1M = pd.DataFrame(df_libro_mayor_ytd2_closed_piv_1M.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_closed_piv_1M  : ", df_libro_mayor_ytd2_closed_piv_1M.shape)

df_libro_mayor_ytd2_closed_piv_1M.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_closed_piv_1M  :  (332, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
21,P-1611,COSTO MATERIALES,6648
80,P-1658,COSTO MANO DE OBRA,0
70,P-1649,COSTO SUPERVISIÓN,0
60,P-1645,COSTO MANO DE OBRA,0
91,P-1661,INGRESO,0
326,P-1727,COSTO SUPERVISIÓN,0
314,P-1724,COSTO SUPERVISIÓN,0
3,1. TOTAL,INGRESO,1520471350
296,P-1720,COSTO MANO DE OBRA,0
312,P-1724,COSTO MANO DE OBRA,-506066


#### 3.1.19 - CREACIÓN DE DATAFRAME BASE LIBRO MAYOR 2023 PARA EL CRUCE OTROS DATAFRAMES

##### NOTA IMPORTANTE
##### Se considera que todos los dataframes que pintan los dashboards deben tener como base los proyectos abiertos en el Libro Mayor 2023. Para esto se crea el dataframe base con el cual debe comenzar cualquier cruce de datos y que considera el código del proyecto, todos los tipos de cuenta por cada proyecto y con valor en cero, además se cruza con la tabla codigo de proyecto.

In [636]:
df_libro_mayor5_base = df_libro_mayor5.copy()

In [637]:
print(df_libro_mayor5_base["CODIGO_PROYECTO"].unique())

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_libro_mayor5_base   : ", df_libro_mayor5_base.shape)

df_libro_mayor5_base = df_libro_mayor5_base[['CODIGO_PROYECTO','TIPO_CUENTA']].drop_duplicates()

# Se elimina fila de TOTAL
df_libro_mayor5_base = df_libro_mayor5_base[df_libro_mayor5_base['CODIGO_PROYECTO'] != '1. TOTAL']

# Se Reemplazan valores de la column TIPO_CUENTA
df_libro_mayor5_base['TIPO_CUENTA'] = df_libro_mayor5_base['TIPO_CUENTA'].replace({
    'INGRESO': '1. INGRESO',
    'COSTO MATERIALES': '2. COSTO MATERIALES',
    'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA',
    'COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'
})

df_libro_mayor5_base['TOTAL_LB']= 0

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)
df_libro_mayor5_base.sample(10)

['P-1557' 'P-1591' 'P-1592' 'P-1603' 'P-1611' 'P-1612' 'P-1620' 'P-1626'
 'P-1632' 'P-1639' 'P-1641' 'P-1642' 'P-1643' 'P-1644' 'P-1645' 'P-1648'
 'P-1649' 'P-1653' 'P-1656' 'P-1658' 'P-1660' 'P-1661' 'P-1663' 'P-1664'
 'P-1666' 'P-1667' 'P-1668' 'P-1669' 'P-1670' 'P-1672' 'P-1673' 'P-1674'
 'P-1675' 'P-1676' 'P-1677' 'P-1679' 'P-1680' 'P-1681' 'P-1682' 'P-1683'
 'P-1684' 'P-1685' 'P-1686' 'P-1687' 'P-1688' 'P-1689' 'P-1690' 'P-1691'
 'P-1692' 'P-1693' 'P-1694' 'P-1695' 'P-1696' 'P-1697' 'P-1698' 'P-1699'
 'P-1700' 'P-1701' 'P-1703' 'P-1704' 'P-1705' 'P-1706' 'P-1707' 'P-1708'
 'P-1709' 'P-1712' 'P-1713' 'P-1714' 'P-1715' 'P-1716' 'P-1717' 'P-1718'
 'P-1719' 'P-1720' 'P-1721' 'P-1722' 'P-1723' 'P-1724' 'P-1725' 'P-1726'
 'P-1727' 'P-1728' '1. TOTAL']
DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_libro_mayor5_base   :  (2515, 9)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (328, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB
1222,P-1672,4. COSTO SUPERVISIÓN,0
119,P-1686,2. COSTO MATERIALES,0
111,P-1684,1. INGRESO,0
346,P-1709,2. COSTO MATERIALES,0
1510,P-1685,4. COSTO SUPERVISIÓN,0
763,P-1641,1. INGRESO,0
2110,P-1713,4. COSTO SUPERVISIÓN,0
598,P-1603,4. COSTO SUPERVISIÓN,0
84,P-1680,1. INGRESO,0
463,P-1724,3. COSTO MANO DE OBRA,0


In [638]:
# Se Agrega COSTO DE SUPERVISIÓN

nueva_fila = pd.DataFrame({'CODIGO_PROYECTO': ['1. TOTAL'],
                           'TIPO_CUENTA': ['4. COSTO SUPERVISIÓN'],
                           'TOTAL_LB': [0]})

df_libro_mayor5_base = pd.concat([df_libro_mayor5_base, nueva_fila], ignore_index=True)

df_libro_mayor5_base[df_libro_mayor5_base["CODIGO_PROYECTO"]=="1. TOTAL"]
print(df_libro_mayor5_base["CODIGO_PROYECTO"].unique())
print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)

['P-1557' 'P-1591' 'P-1592' 'P-1603' 'P-1611' 'P-1612' 'P-1620' 'P-1626'
 'P-1632' 'P-1639' 'P-1641' 'P-1642' 'P-1643' 'P-1644' 'P-1645' 'P-1648'
 'P-1649' 'P-1653' 'P-1656' 'P-1658' 'P-1660' 'P-1661' 'P-1663' 'P-1664'
 'P-1666' 'P-1667' 'P-1668' 'P-1669' 'P-1670' 'P-1672' 'P-1673' 'P-1674'
 'P-1675' 'P-1676' 'P-1677' 'P-1679' 'P-1680' 'P-1681' 'P-1682' 'P-1683'
 'P-1684' 'P-1685' 'P-1686' 'P-1687' 'P-1688' 'P-1689' 'P-1690' 'P-1691'
 'P-1692' 'P-1693' 'P-1694' 'P-1695' 'P-1696' 'P-1697' 'P-1698' 'P-1699'
 'P-1700' 'P-1701' 'P-1703' 'P-1704' 'P-1705' 'P-1706' 'P-1707' 'P-1708'
 'P-1709' 'P-1712' 'P-1713' 'P-1714' 'P-1715' 'P-1716' 'P-1717' 'P-1718'
 'P-1719' 'P-1720' 'P-1721' 'P-1722' 'P-1723' 'P-1724' 'P-1725' 'P-1726'
 'P-1727' 'P-1728' '1. TOTAL']
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (329, 3)


In [639]:
# Crear DataFrame con TIPO_CUENTA y valor cero para cada combinación única

df_nuevo = pd.DataFrame({
    'CODIGO_PROYECTO': df_libro_mayor5_base['CODIGO_PROYECTO'].repeat(4),
    'TIPO_CUENTA': ['1. INGRESO', '2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN'] * len(df_libro_mayor5_base),
    'TOTAL_LB': 0
})

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_nuevo             : ", df_nuevo.shape)

df_libro_mayor5_base = pd.concat([df_libro_mayor5_base, df_nuevo], ignore_index=True)

# Se remueve cualquier duplicado.
df_libro_mayor5_base.drop_duplicates(inplace=True)

print("DIMENSIONES ANTERIOR DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)
df_libro_mayor5_base.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base :  (329, 3)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_nuevo             :  (1316, 3)
DIMENSIONES ANTERIOR DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (332, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB
282,P-1691,4. COSTO SUPERVISIÓN,0
223,P-1660,1. INGRESO,0
287,P-1695,4. COSTO SUPERVISIÓN,0
110,P-1706,3. COSTO MANO DE OBRA,0
163,P-1728,2. COSTO MATERIALES,0


In [640]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5_base.sort_values(by=['CODIGO_PROYECTO','TIPO_CUENTA']).head(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB
1641,1. TOTAL,1. INGRESO,0
1642,1. TOTAL,2. COSTO MATERIALES,0
1643,1. TOTAL,3. COSTO MANO DE OBRA,0
328,1. TOTAL,4. COSTO SUPERVISIÓN,0
164,P-1557,1. INGRESO,0
165,P-1557,2. COSTO MATERIALES,0
0,P-1557,3. COSTO MANO DE OBRA,0
166,P-1557,4. COSTO SUPERVISIÓN,0
167,P-1591,1. INGRESO,0
168,P-1591,2. COSTO MATERIALES,0


In [641]:
# MERGE CON CODIGO_PROYECTO

# Se crea dataframe de codigos de proyecto y se aplica mayúscula
df_cod_proyecto_base = df_cod_proyecto.applymap(lambda s: s.upper() if isinstance(s, str) else s)

# Se hace merge con los codigos de proyecto.

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)

df_libro_mayor5_base2 = pd.merge(df_libro_mayor5_base, df_cod_proyecto_base, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)

df_libro_mayor5_base2.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (332, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (332, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
112,P-1706,1. INGRESO,0,CALIFORNIA,CONSTRUCTORA E INVERSIONES VITAL SPA.
59,P-1686,1. INGRESO,0,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.
197,P-1642,1. INGRESO,0,CORINTO,EMP. CONSTRUCTORA INGENIEROS S.A.
313,P-1719,4. COSTO SUPERVISIÓN,0,CD METALPAR,EBCO S.A.
149,P-1723,1. INGRESO,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.
324,P-1726,4. COSTO SUPERVISIÓN,0,NaN,NaN
298,P-1706,4. COSTO SUPERVISIÓN,0,CALIFORNIA,CONSTRUCTORA E INVERSIONES VITAL SPA.
238,P-1667,1. INGRESO,0,ECO PLACILLA,CONSTRUCTORA ICF SPA
94,P-1699,1. INGRESO,0,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.
55,P-1684,1. INGRESO,0,EDIFICIO GOYCOLEA,EBCO S.A.


In [642]:
# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"

df_libro_mayor5_base2.loc[df_libro_mayor5_base2['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

df_libro_mayor5_base2.sample(10) 

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
330,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL
320,P-1722,4. COSTO SUPERVISIÓN,0,ECO CERRILLOS,CONSTRUCTORA ICF SPA
25,P-1666,2. COSTO MATERIALES,0,CD AXIONLONG,EBCO S.A.
185,P-1626,1. INGRESO,0,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.
171,P-1592,3. COSTO MANO DE OBRA,0,NUEVA COSTANERA 3300,EBCO S.A.
302,P-1712,4. COSTO SUPERVISIÓN,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.
24,P-1664,2. COSTO MATERIALES,0,MK II PCI,COMERCIAL K SPA.
38,P-1676,3. COSTO MANO DE OBRA,0,LO CAMPINO,EBCO S.A.
161,P-1727,1. INGRESO,0,NaN,NaN
315,P-1720,3. COSTO MANO DE OBRA,0,NaN,NaN


In [643]:
# INSPECCIÓN VISUAL 1

df_libro_mayor5_base2[df_libro_mayor5_base2['CODIGO_PROYECTO']=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
328,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL
329,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL
330,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL
331,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL


In [644]:
# INSPECCIÓN VISUAL 2

df_libro_mayor5_base2[df_libro_mayor5_base2['CODIGO_PROYECTO']=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
104,P-1704,3. COSTO MANO DE OBRA,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.
105,P-1704,2. COSTO MATERIALES,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.
106,P-1704,1. INGRESO,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.
296,P-1704,4. COSTO SUPERVISIÓN,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.


### 3.2 - TRATAMIENTO DE DESPACHOS REALIZADOS

#### 3.2.1 - Creación de dataframe copia

In [645]:
df_desp_realizados2 = df_desp_realizados.copy()
df_desp_realizados2.head(5)

,COD_PROYECTO,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL
0,P-1672,VCO,2022-01-19T00:00:00-03:00,44351,1060,T14380048,2405,Cable coaxial RG6,M,143,343915
1,P-1672,VCO,2022-01-19T00:00:00-03:00,44351,1015,T14380056,1300,Cable PE 0.9 x 2,M,184,239200
2,P-1672,VCO,2022-01-19T00:00:00-03:00,44351,1039,T14380057,1500,Cable pin 4x24,M,74,111000
3,P-1672,VCO,2022-01-19T00:00:00-03:00,44351,1039,T14380055,300,Cable paralelo 2x24 Blanco,M,52,15600
4,P-1672,VCO,2022-01-19T00:00:00-03:00,44351,1027,T14381073,200,Cable paralelo 2x18 color,M,124,24800


#### 3.2.2 - Se eliminan proyectos relacionados a STT, P-Proyectos, P-Servicios, P-Mantención

In [646]:
df_desp_realizados2["COD_PROYECTO"].unique()

array(['P-1672', 'P-Servicios', 'P-1691', nan, 'P-Proyectos', 'P-1689',
       'P-1693', 'P-1675', 'P-1667', 'P-1698', 'P-1685', 'P-1701',
       'P-1704', 'P-1676', 'P-ANALISIS PROYE', 'P-1684', 'P-1697',
       'P-Mantención', 'P-1683', 'P-1700', 'P-1661', 'P-1686', 'P-1680',
       'P-1677', 'P-1615', 'P-1668', 'P-1690', 'P-1669', 'P-1648',
       'P-1670', 'P-1694', 'P-1703', 'P-1688', 'P-1712', 'P-PROYECTOS',
       'P-1702', 'P-1645', 'P-1696', 'P-1625', 'P-1681', 'P-1692',
       'P-1682', 'P-1673', 'P-1687', 'P-1714', 'P-1707', 'P-1718',
       'P-1623', 'P-1695', 'P-1715', 'P-1716', 'P-1639', 'P-1721',
       'P-1699', 'P-1705', 'P-1709', 'P-1724', 'P-1664', 'P-1723',
       'P-1728', 'P-1642', 'P-1727', 'P-1726', 'P-1713', 'P-1592',
       'P-1620', 'P-1725', 'P-1652', 'P-1613', 'P-1658', 'P-1638',
       'P-1629', 'P-1663', 'P-1626', 'P-1656', 'P-1660', 'P-1632',
       'P-1636', 'P-1671', 'P-1649', 'P-1653', 'P-1654', 'P-1655',
       'P-1627', 'S-1601', 'P-1678', 'P-1637',

In [647]:
valores_eliminar = ['STT', 'P-Proyectos', 'P-Servicios', 'P-Mantención','P-Oficina''P-ANALISIS PROYE','P-ANALISIS PROYE','P-PROYECTOS','P-Oficina','nan']

# Convertir los valores a eliminar y la columna 'PROYECTO' a mayúsculas
valores_eliminar_mayusculas = [valor.upper() for valor in valores_eliminar]
df_desp_realizados2['PROYECTO_MAYUS'] = df_desp_realizados2['COD_PROYECTO'].str.upper()

# Filtrar las filas que no contengan los valores a eliminar
df_desp_realizados3 = df_desp_realizados2[~df_desp_realizados2['PROYECTO_MAYUS'].isin(valores_eliminar_mayusculas)]

# Eliminar la columna auxiliar 'PROYECTO_MAYUS'
df_desp_realizados3 = df_desp_realizados3.drop(columns=['PROYECTO_MAYUS'])

df_desp_realizados3.sample(10)

,COD_PROYECTO,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL
11921,P-1692,VCO,2022-11-24T00:00:00-03:00,48195,1062,C06110268,83,Splitter distribuidor F 1x2 salida 5-240,UN,738,61254
10210,P-1691,VCO,2022-02-16T00:00:00-03:00,44572,1060,C04082339,2,Bus Control GT BC,UN,30678,61356
19233,P-1715,VCO,2023-04-19T00:00:00-03:00,50530,1156,I15411564,12,"Perno anclaje 1/2"" x 3 3/4""",UN,472,5664
13932,P-1680,VCO,2022-11-18T00:00:00-03:00,48109,1087,I17452714,1,Vaselina Vegetal,UN,7479,7479
4467,P-1723,VCO,2023-04-03T00:00:00-03:00,50166,1102,I16431561,200,Amarras plasticas 200 con banderilla,UN,7,1400
10546,P-1667,VCO,2022-09-01T00:00:00-03:00,47008,1013,I16430010,500,Amarras plasticas 200mm,UN,14,7000
17024,P-1708,VCO,2023-04-21T00:00:00-03:00,50670,1030,S11290198,30,"Magnetico Embutido 1/4"" Blanco",UN,750,22500
5005,P-1613,GDP,2022-03-31T00:00:00-03:00,44901,1020,S11280105,4,Centralita Bosch DS3MX,UN,19700,78800
11009,P-1667,VCO,2022-11-03T00:00:00-03:00,47853,1048,P08180790,50,Gabinete Extintor PQS 6 Kg Sobreponer pu,UN,29994,1499700
6589,P-1718,VCO,2023-04-20T00:00:00-03:00,50626,1140,I15411619,2,Perfil angulo laminado 40x40x5mm,T6M,18745,37490


#### 3.2.2 - Se eliminan Tipos de movimientos diferentes a VCO y GDP

In [648]:
# Reemplazar los valores NaN en la columna "TIPO MOVIMIENTO" con una cadena vacía
df_desp_realizados3['TIPO_x0020_MOVIMIENTO'] = df_desp_realizados3['TIPO_x0020_MOVIMIENTO'].fillna('')

# Filtrar las filas donde "TIPO MOVIMIENTO" sea igual a "VCO" o "GDP" y excluir otros valores

mask = df_desp_realizados3['TIPO_x0020_MOVIMIENTO'].isin(['VCO', 'GDP'])
df_desp_realizados3 = df_desp_realizados3[mask]

In [649]:
# Se realiza la evaluación de dimensiones después de la eliminación de filas
print("\nDataframe Original antes de los cambios:")
print(df_desp_realizados.shape)
print("Dataframe después de eliminar registros:")
print(df_desp_realizados3.shape)

df_desp_realizados3.to_csv('df_desp_realizados3.csv', index = False)


Dataframe Original antes de los cambios:
(19815, 11)
Dataframe después de eliminar registros:
(11736, 11)


In [650]:
# Convertir la columna 'FECHA GUIA' a objetos de fecha y hora, manejando campos en blanco o no válidos como fechas
df_desp_realizados3['FECHA GUIA 2'] = pd.to_datetime(df_desp_realizados3['FECHA_x0020_GUIA'], errors='coerce')

# Extraer el MES y el AÑO en nuevas columnas utilizando funciones lambda y apply()
df_desp_realizados3['MES'] = df_desp_realizados3['FECHA GUIA 2'].apply(lambda x: x.month if pd.notnull(x) else None)
df_desp_realizados3['AÑO'] = df_desp_realizados3['FECHA GUIA 2'].apply(lambda x: x.year if pd.notnull(x) else None)

df_desp_realizados3.sample(5)

,COD_PROYECTO,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL,FECHA GUIA 2,MES,AÑO
2859,P-1693,GDP,2023-01-09T00:00:00-03:00,48813,1013,T14372591,3,"Conduit galvanizado EMT 25 x3000mm (3/4"")",T3M,4162,12486,2023-01-09 00:00:00-03:00,1,2023
13695,P-1690,VCO,2023-02-13T00:00:00-03:00,49419,1001,S09232564,2,DVR 16CH Hikvision HDCVI DS-7216HGHI-k1,UN,51072,102144,2023-02-13 00:00:00-03:00,2,2023
16841,P-1695,VCO,2023-06-06T00:00:00-04:00,51552,1046,C06112917,4,SOPORTE MASTIL L ZINC+CPR ATORNILLABLE 3,UN,5720,22880,2023-06-06 00:00:00-04:00,6,2023
6647,P-1667,VCO,2022-01-10T00:00:00-03:00,44245,1023,I17440475,2,"Hoja se sierra de 1/2 x 24 d 12""",UN,899,1798,2022-01-10 00:00:00-03:00,1,2022
9814,P-1685,VCO,2022-04-06T00:00:00-03:00,44953,1143,P08202650,15,"Union copla 2"" ranurada rigida",UN,1987,29805,2022-04-06 00:00:00-03:00,4,2022


#### 3.2.4 - Se realizan tablas pivot

In [651]:
df_desp_realizados3['COSTO_x0020_TOTAL'] = df_desp_realizados3['COSTO_x0020_TOTAL'].astype(int)
df_desp_realizados3['CANTIDAD'] = df_desp_realizados3['CANTIDAD'].astype(float)

In [652]:
df_desp_realizados3.columns

Index(['COD_PROYECTO', 'TIPO_x0020_MOVIMIENTO', 'FECHA_x0020_GUIA',
       'NUMERO_x0020_GUIA', 'PARTIDA_x0020_PRESUPUESTARIA',
       'CODIGO_x0020_PARTE', 'CANTIDAD', 'DESCRIPCI_xFFFD_N_x0020_PRODUCTO',
       'UNIDAD_x0020_DE_x0020_MEDIDA', 'COSTO_x0020_UNITARIO',
       'COSTO_x0020_TOTAL', 'FECHA GUIA 2', 'MES', 'AÑO'],
      dtype='object')

In [653]:
# Crear el pivot dataframe
df_desp_realizados4 = pd.pivot_table(df_desp_realizados3, 
                          values=['CANTIDAD', 'COSTO_x0020_TOTAL'], 
                          index=['COD_PROYECTO', 'AÑO', 'MES'], 
                          aggfunc='sum', 
                          columns='TIPO_x0020_MOVIMIENTO')

df_desp_realizados4 = pd.DataFrame(df_desp_realizados4.to_records())

df_desp_realizados4.sample(10)

,COD_PROYECTO,AÑO,MES,"('CANTIDAD', 'GDP')","('CANTIDAD', 'VCO')","('COSTO_x0020_TOTAL', 'GDP')","('COSTO_x0020_TOTAL', 'VCO')"
412,P-1698,2022,7,9.00,195.00,29142.00,3130139.00
190,P-1675,2023,6,4.00,NaN,27469.00,NaN
321,P-1690,2022,8,NaN,2220.00,NaN,2032032.00
193,P-1676,2022,3,NaN,3961.00,NaN,1170165.00
80,P-1654,2022,5,200.00,NaN,10400.00,NaN
545,P-1716,2023,5,NaN,896.00,NaN,1481523.00
566,P-1724,2023,3,NaN,476.00,NaN,2449255.00
262,P-1684,2023,2,NaN,2175.00,NaN,437948.00
59,P-1648,2022,7,17.00,NaN,19951.00,NaN
408,P-1697,2023,6,NaN,262.00,NaN,774061.00


#### 3.2.5 - Se renombran columnas 

In [654]:
df_desp_realizados4 = df_desp_realizados4.rename(columns={'COD_PROYECTO': 'CODIGO_PROYECTO'})
df_desp_realizados4 = df_desp_realizados4.rename(columns={"('CANTIDAD', 'GDP')": "CANT. GDP"})
df_desp_realizados4 = df_desp_realizados4.rename(columns={"('CANTIDAD', 'VCO')": "CANT. VCO"})
df_desp_realizados4 = df_desp_realizados4.rename(columns={"('COSTO_x0020_TOTAL', 'GDP')": "TOTAL. GDP"})
df_desp_realizados4 = df_desp_realizados4.rename(columns={"('COSTO_x0020_TOTAL', 'VCO')": "TOTAL. VCO"})

df_desp_realizados4.sample(5)

,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO
407,P-1697,2023,5,NaN,263.00,NaN,2793616.00
119,P-1664,2023,2,10.00,NaN,82955.00,NaN
347,P-1692,2022,10,NaN,2789.00,NaN,3281448.00
431,P-1700,2022,12,45.00,545.00,74718.00,3436341.00
400,P-1697,2022,9,NaN,4405.00,NaN,871004.00


#### 3.2.6 - Se reemplazas columnas NaN por ceros y se crea columna de TOTAL GPD-VCO

In [655]:
df_desp_realizados4.fillna(0, inplace=True)
df_desp_realizados4['TOTAL. VCO - GDP'] = df_desp_realizados4['TOTAL. VCO'] - df_desp_realizados4['TOTAL. GDP']
df_desp_realizados4.sample(5)

,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP
5,P-1592,2023,2,0.00,1.00,0.00,102993.00,102993.00
212,P-1677,2022,10,0.00,2035.00,0.00,8980005.00,8980005.00
290,P-1687,2022,5,0.00,5.00,0.00,72772.00,72772.00
45,P-1644,2022,9,1.00,0.00,22500.00,0.00,-22500.00
287,P-1686,2023,4,0.00,2420.00,0.00,4886851.00,4886851.00


#### 3.2.7 - Se seleccionan solamente los registros del mes y año en curso 

In [656]:
# Obtener el año y mes actual
#now = datetime.datetime.now()

now = datetime.now()
current_year = now.year
print(current_year)
current_month = now.month
print(current_month)
print("Dimensiones antes del filtro", df_desp_realizados4.shape)

# Filtrar el dataframe por año y mes
df_desp_realizados5 = df_desp_realizados4[(df_desp_realizados4['AÑO'] == current_year) & (df_desp_realizados4['MES'] == current_month)]
print("Dimensiones después del filtro", df_desp_realizados5.shape)
df_desp_realizados5.sample(5)

2023
6
Dimensiones antes del filtro (584, 8)
Dimensiones después del filtro (27, 8)


,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP
289,P-1686,2023,6,0.00,13.00,0.00,390181.00,390181.00
109,P-1663,2023,6,7.00,0.00,5429.00,0.00,-5429.00
500,P-1708,2023,6,0.00,2.00,0.00,45820.00,45820.00
376,P-1694,2023,6,0.00,8462.00,0.00,5722102.00,5722102.00
247,P-1682,2023,6,3.00,0.00,38788.00,0.00,-38788.00


#### 3.2.7-2 - Se seleccionan solamente los registros del mes y año en curso 2
Se considera esta línea para poblar la vista del mes anterior.

In [657]:
# Obtener el año y mes actual
#now = datetime.datetime.now()

now = datetime.now()
current_year = now.year
print(current_year)
current_month = now.month
print(current_month)
print("Dimensiones antes del filtro", df_desp_realizados4.shape)

mes_anterior = (datetime.now() - timedelta(days=30)).month

# Filtrar el dataframe por año y mes
df_desp_realizados5_1M = df_desp_realizados4[(df_desp_realizados4['AÑO'] == current_year) & (df_desp_realizados4['MES'] == mes_anterior)]
print("Dimensiones después del filtro", df_desp_realizados5_1M.shape)
df_desp_realizados5_1M.sample(5)

2023
6
Dimensiones antes del filtro (584, 8)
Dimensiones después del filtro (46, 8)


,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP
436,P-1700,2023,5,52.00,60.00,13104.00,38040.00,24936.00
72,P-1649,2023,5,1.00,0.00,36183.00,0.00,-36183.00
31,P-1632,2023,5,15.00,0.00,75965.00,0.00,-75965.00
233,P-1680,2023,5,0.00,6108.00,0.00,11122532.00,11122532.00
135,P-1667,2023,5,0.00,513.00,0.00,90852.00,90852.00


#### 3.2.8 - Se agrega la fila TOTAL

In [658]:
# Obtener la suma de cada columna y guardarla en un diccionario

print("Dimensiones antes del filtro", df_desp_realizados5.shape)

total_row = {
    'CODIGO_PROYECTO': 'TOTAL',
    'AÑO': df_desp_realizados5['AÑO'].iloc[0], # Obtener el valor del año de la primera fila
    'MES': df_desp_realizados5['MES'].iloc[0], # Obtener el valor del mes de la primera fila
    'CANT. GDP': df_desp_realizados5['CANT. GDP'].sum(),
    'CANT. VCO': df_desp_realizados5['CANT. VCO'].sum(),
    'TOTAL. GDP': df_desp_realizados5['TOTAL. GDP'].sum(),
    'TOTAL. VCO': df_desp_realizados5['TOTAL. VCO'].sum(),
    'TOTAL. VCO - GDP': df_desp_realizados5['TOTAL. VCO - GDP'].sum()
}

total_df = pd.DataFrame(total_row, index=[0]) # Crear un nuevo DataFrame con la fila 'TOTAL'
df_desp_realizados5 = pd.concat([df_desp_realizados5, total_df], ignore_index=True) # Concatenar el nuevo DataFrame con el original

#df_desp_realizados5["TIPO_CUENTA"]= df_desp_realizados5["TOTAL. VCO - GDP"]
df_desp_realizados5["TOTAL"]= df_desp_realizados5["TOTAL. VCO - GDP"]
df_desp_realizados5["TIPO_CUENTA"]= "COSTO MATERIALES"

df_desp_realizados5.sample()

print("Dimensiones después del filtro", df_desp_realizados5.shape)

df_desp_realizados5.sample(5)

Dimensiones antes del filtro (27, 8)
Dimensiones después del filtro (28, 10)


,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP,TOTAL,TIPO_CUENTA
25,P-1726,2023,6,0.00,20.00,0.00,71216.00,71216.00,71216.00,COSTO MATERIALES
24,P-1725,2023,6,78.00,9784.00,1280790.00,4887909.00,3607119.00,3607119.00,COSTO MATERIALES
6,P-1694,2023,6,0.00,8462.00,0.00,5722102.00,5722102.00,5722102.00,COSTO MATERIALES
7,P-1695,2023,6,0.00,853.00,0.00,1622389.00,1622389.00,1622389.00,COSTO MATERIALES
8,P-1697,2023,6,0.00,262.00,0.00,774061.00,774061.00,774061.00,COSTO MATERIALES


#### 3.2.8-2 - Se agrega la fila TOTAL 2

In [659]:
# Obtener la suma de cada columna y guardarla en un diccionario

print("Dimensiones antes del filtro", df_desp_realizados5_1M.shape)

total_row = {
    'CODIGO_PROYECTO': 'TOTAL',
    'AÑO': df_desp_realizados5_1M['AÑO'].iloc[0], # Obtener el valor del año de la primera fila
    'MES': df_desp_realizados5_1M['MES'].iloc[0], # Obtener el valor del mes de la primera fila
    'CANT. GDP': df_desp_realizados5_1M['CANT. GDP'].sum(),
    'CANT. VCO': df_desp_realizados5_1M['CANT. VCO'].sum(),
    'TOTAL. GDP': df_desp_realizados5_1M['TOTAL. GDP'].sum(),
    'TOTAL. VCO': df_desp_realizados5_1M['TOTAL. VCO'].sum(),
    'TOTAL. VCO - GDP': df_desp_realizados5_1M['TOTAL. VCO - GDP'].sum()
}

total_df = pd.DataFrame(total_row, index=[0]) # Crear un nuevo DataFrame con la fila 'TOTAL'
df_desp_realizados5_1M = pd.concat([df_desp_realizados5_1M, total_df], ignore_index=True) # Concatenar el nuevo DataFrame con el original

#df_desp_realizados5["TIPO_CUENTA"]= df_desp_realizados5["TOTAL. VCO - GDP"]
df_desp_realizados5_1M["TOTAL"]= df_desp_realizados5_1M["TOTAL. VCO - GDP"]
df_desp_realizados5_1M["TIPO_CUENTA"]= "COSTO MATERIALES"

print("Dimensiones después del filtro", df_desp_realizados5_1M.shape)

df_desp_realizados5_1M.sample(5)

Dimensiones antes del filtro (46, 8)
Dimensiones después del filtro (47, 10)


,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP,TOTAL,TIPO_CUENTA
23,P-1696,2023,5,0.00,1.00,0.00,2304440.00,2304440.00,2304440.00,COSTO MATERIALES
41,P-1723,2023,5,18.00,62814.00,3285765.00,37324477.00,34038712.00,34038712.00,COSTO MATERIALES
9,P-1669,2023,5,7.00,0.00,245707.00,0.00,-245707.00,-245707.00,COSTO MATERIALES
46,TOTAL,2023,5,800.00,160715.00,8533707.00,174983604.00,166449897.00,166449897.00,COSTO MATERIALES
32,P-1707,2023,5,12.00,6235.00,257844.00,5892447.00,5634603.00,5634603.00,COSTO MATERIALES


In [660]:
df_desp_realizados5.to_csv('archivos_respaldo/df_desp_realizados5.csv', mode='a')
df_desp_realizados5_1M.to_csv('archivos_respaldo/df_desp_realizados5_1M.csv', mode='a')

### 3.4 TRATAMIENTO "PROYECTADO"

#### 3.4.1 CREACIÓN NUEVO DATAFRAME DE COPIA.

In [661]:
df_proyectado2 = df_proyectado
df_proyectado2.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 3.4.1-2 CREACIÓN NUEVO DATAFRAME DE COPIA 2.

In [662]:
df_proyectado2_1M = df_proyectado_1M
df_proyectado2_1M.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 3.4.2 LIMPIEZA DE DATAFRAME Y SELECCIÓN DE COLUMNAS UTILES

In [663]:
df_proyectado2 = df_proyectado2.drop(range(0, 6))
new_header = df_proyectado2.iloc[0]
df_proyectado2 = df_proyectado2[1:]
df_proyectado2.columns = new_header
df_proyectado2 = df_proyectado2.loc[:, ["PROYECTO", "PROYECTO2", "INGRESO","MATERIALES","MANO DE OBRA"]]
df_proyectado2.sample(10)

6,PROYECTO,PROYECTO2,INGRESO,MATERIALES,MANO DE OBRA
23,P-1668,QUILLAYES,NaN,NaN,NaN
28,P-1674,Beiersdorf,NaN,NaN,NaN
87,NaN,NaN,12000000,NaN,NaN
12,P-1632,CHORILLOS CLOROX,NaN,NaN,NaN
46,P-1693,VOLTA,NaN,NaN,NaN
73,P-1722,ECO CERRILLO,NaN,NaN,NaN
21,P-1663,Oficinas Equifax,NaN,NaN,NaN
58,P-1705,Los Clarines,10500000,4500000,NaN
39,P-1686,ICALMA,5000000,2000000,NaN
19,P-1660,CORONEL ALVARADO,NaN,NaN,NaN


#### 3.4.2-2 LIMPIEZA DE DATAFRAME Y SELECCIÓN DE COLUMNAS UTILES 2

In [664]:
df_proyectado2_1M = df_proyectado2_1M.drop(range(0, 6))
new_header = df_proyectado2_1M.iloc[0]
df_proyectado2_1M = df_proyectado2_1M[1:]
df_proyectado2_1M.columns = new_header
df_proyectado2_1M = df_proyectado2_1M.loc[:, ["PROYECTO", "PROYECTO2", "INGRESO","MATERIALES","MANO DE OBRA"]]
df_proyectado2_1M.sample(10)

6,PROYECTO,PROYECTO2,INGRESO,MATERIALES,MANO DE OBRA
31,P-1677,GALVARINO,NaN,NaN,NaN
67,P-1716,Astara,10082518,NaN,NaN
17,P-1655,DECATHLON,NaN,NaN,NaN
11,P-1652,ABDON CIFUENTES,NaN,NaN,NaN
47,P-1694,ECO HUEMUL,17000065,7000000,NaN
68,P-1717,ECO QUILIN,NaN,NaN,NaN
57,P-1704,MERCADO LIBRE,68606659.28,24000000,NaN
50,P-1697,FORTALEZA ESTOCOLMO,8945493.92,2500000,NaN
78,P-1727,BUDNIK,17255460,9500000,NaN
63,P-1712,CC VIÑA Outlet Park,15036000,8500000,NaN


#### 3.4.3 TRATAMIENTO DE DATOS NAN

In [665]:
# seleccionar las columnas deseadas
df_proyectado3 = df_proyectado2[["PROYECTO", "PROYECTO2", "INGRESO","MATERIALES","MANO DE OBRA"]]
# eliminar la columna índice
df_proyectado3 = df_proyectado3.reset_index(drop=True)
# eliminar las filas que contengan NaN en la columna P
df_proyectado3 = df_proyectado3.dropna(subset=['PROYECTO'])
# reemplazar los valores NaN en la columna INGRESO por cero
df_proyectado3['INGRESO'] = df_proyectado3['INGRESO'].fillna(0)
df_proyectado3['MATERIALES'] = df_proyectado3['MATERIALES'].fillna(0)
df_proyectado3['MANO DE OBRA'] = df_proyectado3['MANO DE OBRA'].fillna(0)

# renombrar la columna P a CODIGO_PROYECTO
df_proyectado3 = df_proyectado3.rename(columns={'PROYECTO': 'CODIGO_PROYECTO'})
df_proyectado3 = df_proyectado3.rename(columns={'PROYECTO2': 'PROYECTO'})

# poner en mayúscula el contenido de las columnas CODIGO_PROYECTO y PROYECTO
df_proyectado3['CODIGO_PROYECTO'] = df_proyectado3['CODIGO_PROYECTO'].str.upper()
df_proyectado3['PROYECTO'] = df_proyectado3['PROYECTO'].str.upper()

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3  : ", df_proyectado3.shape)

df_proyectado3.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3  :  (74, 5)


6,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA
9,P-1654,EXEQUIEL FERNANDEZ 1670,0,0,0
17,P-1669,DEPARTAMENTAL II,0,0,0
43,P-1697,FORTALEZA ESTOCOLMO,5000000,1000000,0
54,P-1708,ECO ESPAÑA,9500000,4200000,0
28,P-1682,PLAZA CEPPI,0,0,0
21,P-1674,BEIERSDORF,0,0,0
8,P-1645,LOS TRES ANTONIO,0,0,0
68,P-1724,185 VOLTA,0,0,0
2,P-1615,ECO SANTIAGO,0,0,0
7,P-1642,CORINTOS,0,0,0


In [666]:
print(df_proyectado3.columns)

Index(['CODIGO_PROYECTO', 'PROYECTO', 'INGRESO', 'MATERIALES', 'MANO DE OBRA'], dtype='object', name=6)


In [667]:
# Eliminar la columna '6'
df_proyectado3 = df_proyectado3.rename_axis(None, axis=1)
df_proyectado3 = df_proyectado3.dropna(subset=['CODIGO_PROYECTO'])

print(df_proyectado3.shape)
df_proyectado3.sample(15)

(72, 5)


,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA
3,P-1621,RENAULT,0,0,0
55,P-1709,CC PASEO SAN PEDRO,0,0,0
23,P-1676,PORTAL LO CAMPINO,0,0,0
1,P-1601,CUATRE CASAS,0,0,0
51,P-1705,LOS CLARINES,10500000,4500000,0
46,P-1700,ORIZON,0,0,0
56,P-1712,CC VIÑA OUTLET PARK,12500000,7700000,0
18,P-1670,MK SEGURIDAS,0,0,0
48,P-1702,KARUN VITACURA,0,0,0
57,P-1713,51 EDIFICIO RESERVA SAN FRANCISCO,16500000,6000000,0


#### 3.4.3-2 TRATAMIENTO DE DATOS NAN 2

In [668]:
# seleccionar las columnas deseadas
df_proyectado3_1M = df_proyectado2_1M[["PROYECTO", "PROYECTO2", "INGRESO","MATERIALES","MANO DE OBRA"]]
# eliminar la columna índice
df_proyectado3_1M = df_proyectado3_1M.reset_index(drop=True)
# eliminar las filas que contengan NaN en la columna P
df_proyectado3_1M = df_proyectado3_1M.dropna(subset=['PROYECTO'])
# reemplazar los valores NaN en la columna INGRESO por cero
df_proyectado3_1M['INGRESO'] = df_proyectado3_1M['INGRESO'].fillna(0)
df_proyectado3_1M['MATERIALES'] = df_proyectado3_1M['MATERIALES'].fillna(0)
df_proyectado3_1M['MANO DE OBRA'] = df_proyectado3_1M['MANO DE OBRA'].fillna(0)

# renombrar la columna P a CODIGO_PROYECTO
df_proyectado3_1M = df_proyectado3_1M.rename(columns={'PROYECTO': 'CODIGO_PROYECTO'})
df_proyectado3_1M = df_proyectado3_1M.rename(columns={'PROYECTO2': 'PROYECTO'})

# poner en mayúscula el contenido de las columnas CODIGO_PROYECTO y PROYECTO
df_proyectado3_1M['CODIGO_PROYECTO'] = df_proyectado3_1M['CODIGO_PROYECTO'].str.upper()
df_proyectado3_1M['PROYECTO'] = df_proyectado3_1M['PROYECTO'].str.upper()

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3_1M  : ", df_proyectado3_1M.shape)

df_proyectado3_1M.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3_1M  :  (73, 5)


6,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA
59,P-1715,LO ECHEVERS,17542000.00,6000000,0
63,P-1719,METALPARK,0.00,0,0
36,P-1690,ONGOLMO,0.00,1500000,0
45,P-1699,VILLA PANAMERICANA LOTE B,21598229.00,18000000,0
44,P-1698,PCI ANDES,0.00,0,0
32,P-1686,ICALMA,10669236.17,4000000,0
7,P-1642,CORINTOS,0.00,0,0
31,NaN,MAPA,0.00,0,0
43,P-1697,FORTALEZA ESTOCOLMO,8945493.92,2500000,0
47,P-1701,ZENTENO,0.00,0,0


#### 3.4.4 SE AGREGA LA COLUMNA TOTAL PARA EL DATAFRAME Y SE FILTRA POR EL MES EN CURSO

In [669]:
# Obtener el año y mes actual

import datetime

current_year = datetime.datetime.now().year
current_month = datetime.datetime.now().month

# Calcula la sumatoria de la columna 'INGRESO'
total_ingreso = df_proyectado3['INGRESO'].sum()
total_materiales = df_proyectado3['MATERIALES'].sum()
total_mano_obra = df_proyectado3['MANO DE OBRA'].sum()

# Crea la fila 'TOTAL'
total_row = {'CODIGO_PROYECTO': 'TOTAL', 'PROYECTO': '', 'INGRESO': total_ingreso,'MATERIALES': total_materiales,'MANO DE OBRA': total_mano_obra}

# Crea un DataFrame con la fila 'TOTAL'
df_total = pd.DataFrame([total_row])

# Concatena el DataFrame original con el DataFrame que contiene la fila 'TOTAL'
df_proyectado3 = pd.concat([df_proyectado3, df_total], ignore_index=True)

#Agreación de columnas para estandarización
df_proyectado3["TOTAL"] = df_proyectado3["INGRESO"]
df_proyectado3["TIPO_CUENTA"] = "INGRESO"
df_proyectado3["MES"] = current_month 
df_proyectado3["AÑO"] = current_year 
df_proyectado3["ORIGEN"] = "PROYECTADO" 

# Se reemplazan valores NaN en 0 
df_proyectado3 = df_proyectado3.fillna(value=0)

# Multiplicar las columnas MATERIALES y MANO DE OBRA por -1 para obtener valores negativos
df_proyectado3[['MATERIALES', 'MANO DE OBRA']] = df_proyectado3[['MATERIALES', 'MANO DE OBRA']].mul(-1)

# Se calcula la columna TOTAL como la suma de INGRESO, MATERIALES y MANO DE OBRA
df_proyectado3['TOTAL'] = df_proyectado3[['INGRESO', 'MATERIALES', 'MANO DE OBRA']].sum(axis=1)

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3  : ", df_proyectado3.shape)
df_proyectado3.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3  :  (73, 10)


,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA,TOTAL,TIPO_CUENTA,MES,AÑO,ORIGEN
32,P-1688,SUIZA,2500000,-1000000,0,1500000,INGRESO,6,2023,PROYECTADO
35,P-1691,ROSARIO ROSALES,0,0,0,0,INGRESO,6,2023,PROYECTADO
2,P-1615,ECO SANTIAGO,0,0,0,0,INGRESO,6,2023,PROYECTADO
72,TOTAL,,449100000,-176535806,0,272564194,INGRESO,6,2023,PROYECTADO
26,P-1679,JUAN SABAJ,0,0,0,0,INGRESO,6,2023,PROYECTADO


#### 3.4.4-2 SE AGREGA LA COLUMNA TOTAL PARA EL DATAFRAME Y SE FILTRA POR EL MES EN CURSO 2

In [670]:
# Obtener el año y mes actual

import datetime

current_year = datetime.datetime.now().year
current_month = datetime.datetime.now().month

# Calcula la sumatoria de la columna 'INGRESO'
total_ingreso = df_proyectado3_1M['INGRESO'].sum()
total_materiales = df_proyectado3_1M['MATERIALES'].sum()
total_mano_obra = df_proyectado3_1M['MANO DE OBRA'].sum()

# Crea la fila 'TOTAL'
total_row = {'CODIGO_PROYECTO': 'TOTAL', 'PROYECTO': '', 'INGRESO': total_ingreso,'MATERIALES': total_materiales,'MANO DE OBRA': total_mano_obra}

# Crea un DataFrame con la fila 'TOTAL'
df_total = pd.DataFrame([total_row])

# Concatena el DataFrame original con el DataFrame que contiene la fila 'TOTAL'
df_proyectado3_1M = pd.concat([df_proyectado3, df_total], ignore_index=True)

#Agreación de columnas para estandarización
df_proyectado3_1M["TOTAL"] = df_proyectado3["INGRESO"]
df_proyectado3_1M["TIPO_CUENTA"] = "INGRESO"
df_proyectado3_1M["MES"] = current_month 
df_proyectado3_1M["AÑO"] = current_year 
df_proyectado3_1M["ORIGEN"] = "PROYECTADO" 

# Se reemplazan valores NaN en 0 
df_proyectado3_1M = df_proyectado3_1M.fillna(value=0)

# Multiplicar las columnas MATERIALES y MANO DE OBRA por -1 para obtener valores negativos
df_proyectado3_1M[['MATERIALES', 'MANO DE OBRA']] = df_proyectado3_1M[['MATERIALES', 'MANO DE OBRA']].mul(-1)

# Se calcula la columna TOTAL como la suma de INGRESO, MATERIALES y MANO DE OBRA
df_proyectado3_1M['TOTAL'] = df_proyectado3_1M[['INGRESO', 'MATERIALES', 'MANO DE OBRA']].sum(axis=1)

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3_1M  : ", df_proyectado3_1M.shape)
df_proyectado3_1M.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3_1M  :  (74, 10)


,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA,TOTAL,TIPO_CUENTA,MES,AÑO,ORIGEN
55,P-1713,51 EDIFICIO RESERVA SAN FRANCISCO,16500000.00,6000000,0,22500000.00,INGRESO,6,2023,PROYECTADO
34,P-1690,ONGOLMO,5000000.00,2000000,0,7000000.00,INGRESO,6,2023,PROYECTADO
66,P-1724,185 VOLTA,0.00,0,0,0.00,INGRESO,6,2023,PROYECTADO
42,P-1698,PCI ANDES,0.00,0,0,0.00,INGRESO,6,2023,PROYECTADO
49,P-1705,LOS CLARINES,10500000.00,4500000,0,15000000.00,INGRESO,6,2023,PROYECTADO


#### 3.4.5 ESTRATEGIA PARA TRANSFORMAR LAS COLUMNAS DE MANO DE OBRA, MATERIALES E INGRESOS EN FILAS. 

In [671]:
print("Dimensiones solo df_proyectado3 antes de cambios",df_proyectado3.shape)

# Crear el subDataFrame sin las columnas MANO DE OBRA, MATERIALES, TOTAL y TIPO_CUENTA
df_sub = df_proyectado3.drop(['MANO DE OBRA', 'MATERIALES', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor INGRESO
df_sub['TIPO_CUENTA'] = 'INGRESO'

# Renombrar la columna INGRESO como TOTAL
df_sub = df_sub.rename(columns={'INGRESO': 'TOTAL'})
print("Dimensiones solo INGRESO",df_sub.shape)

# Crear el subDataFrame sin las columnas INGRESO, MATERIALES, TOTAL y TIPO_CUENTA
df_sub2 = df_proyectado3.drop(['INGRESO', 'MATERIALES', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor MANO DE OBRA
df_sub2['TIPO_CUENTA'] = 'MANO DE OBRA'

# Renombrar la columna MANO DE OBRA como TOTAL
df_sub2 = df_sub2.rename(columns={'MANO DE OBRA': 'TOTAL'})
print("Dimensiones solo MANO DE OBRA",df_sub2.shape)


# Crear el subDataFrame sin las columnas INGRESO, MANO DE OBRA, TOTAL y TIPO_CUENTA
df_sub3 = df_proyectado3.drop(['INGRESO', 'MANO DE OBRA', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor MANO DE OBRA
df_sub3['TIPO_CUENTA'] = 'MATERIALES'

# Renombrar la columna MANO DE OBRA como TOTAL
df_sub3 = df_sub3.rename(columns={'MATERIALES': 'TOTAL'})
print("Dimensiones solo MATERIALES",df_sub3.shape)

# Concatenar los DataFrames verticalmente
df_proyectado4 = pd.concat([df_sub, df_sub2, df_sub3])

# Mostrar el nuevo DataFrame concatenado

print("Dimensiones solo df_proyectado4 después de cambios",df_proyectado4.shape)

df_proyectado4.sample(10)

Dimensiones solo df_proyectado3 antes de cambios (73, 10)
Dimensiones solo INGRESO (73, 7)
Dimensiones solo MANO DE OBRA (73, 7)
Dimensiones solo MATERIALES (73, 7)
Dimensiones solo df_proyectado4 después de cambios (219, 7)


,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
14,P-1663,OFICINAS EQUIFAX,0,6,2023,PROYECTADO,MANO DE OBRA
52,P-1708,ECO ESPAÑA,-4200000,6,2023,PROYECTADO,MATERIALES
36,P-1692,JOSE PEREZ COTAPO,6800000,6,2023,PROYECTADO,INGRESO
46,P-1702,KARUN VITACURA,0,6,2023,PROYECTADO,MATERIALES
45,P-1701,ZENTENO,0,6,2023,PROYECTADO,MATERIALES
39,P-1695,ESTORIL,9000000,6,2023,PROYECTADO,INGRESO
25,P-1678,PIRER,0,6,2023,PROYECTADO,MANO DE OBRA
35,P-1691,ROSARIO ROSALES,0,6,2023,PROYECTADO,MATERIALES
39,P-1695,ESTORIL,-3000000,6,2023,PROYECTADO,MATERIALES
30,P-1686,ICALMA,0,6,2023,PROYECTADO,MANO DE OBRA


In [672]:
#### 3.4.5-2 ESTRATEGIA PARA TRANSFORMAR LAS COLUMNAS DE MANO DE OBRA, MATERIALES E INGRESOS EN FILAS 2  

In [673]:
print("Dimensiones solo df_proyectado3 antes de cambios",df_proyectado3_1M.shape)

# Crear el subDataFrame sin las columnas MANO DE OBRA, MATERIALES, TOTAL y TIPO_CUENTA
df_sub = df_proyectado3_1M.drop(['MANO DE OBRA', 'MATERIALES', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor INGRESO
df_sub['TIPO_CUENTA'] = 'INGRESO'

# Renombrar la columna INGRESO como TOTAL
df_sub = df_sub.rename(columns={'INGRESO': 'TOTAL'})
print("Dimensiones solo INGRESO",df_sub.shape)

# Crear el subDataFrame sin las columnas INGRESO, MATERIALES, TOTAL y TIPO_CUENTA
df_sub2 = df_proyectado3_1M.drop(['INGRESO', 'MATERIALES', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor MANO DE OBRA
df_sub2['TIPO_CUENTA'] = 'MANO DE OBRA'

# Renombrar la columna MANO DE OBRA como TOTAL
df_sub2 = df_sub2.rename(columns={'MANO DE OBRA': 'TOTAL'})
print("Dimensiones solo MANO DE OBRA",df_sub2.shape)


# Crear el subDataFrame sin las columnas INGRESO, MANO DE OBRA, TOTAL y TIPO_CUENTA
df_sub3 = df_proyectado3_1M.drop(['INGRESO', 'MANO DE OBRA', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor MANO DE OBRA
df_sub3['TIPO_CUENTA'] = 'MATERIALES'

# Renombrar la columna MANO DE OBRA como TOTAL
df_sub3 = df_sub3.rename(columns={'MATERIALES': 'TOTAL'})
print("Dimensiones solo MATERIALES",df_sub3.shape)

# Concatenar los DataFrames verticalmente
df_proyectado4_1M = pd.concat([df_sub, df_sub2, df_sub3])

# Mostrar el nuevo DataFrame concatenado

print("Dimensiones solo df_proyectado4 después de cambios",df_proyectado4_1M.shape)

df_proyectado4_1M.sample(10)

Dimensiones solo df_proyectado3 antes de cambios (74, 10)
Dimensiones solo INGRESO (74, 7)
Dimensiones solo MANO DE OBRA (74, 7)
Dimensiones solo MATERIALES (74, 7)
Dimensiones solo df_proyectado4 después de cambios (222, 7)


,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
56,P-1714,DEPARTAMENTAL II,0.00,6,2023,PROYECTADO,MANO DE OBRA
12,P-1660,CORONEL ALVARADO,0.00,6,2023,PROYECTADO,MATERIALES
21,P-1674,BEIERSDORF,0.00,6,2023,PROYECTADO,MANO DE OBRA
24,P-1677,GALVARINO,0.00,6,2023,PROYECTADO,INGRESO
28,P-1682,PLAZA CEPPI,0.00,6,2023,PROYECTADO,MANO DE OBRA
9,P-1654,EXEQUIEL FERNANDEZ 1670,0.00,6,2023,PROYECTADO,MATERIALES
29,P-1684,GOYCOLEA,25000000.00,6,2023,PROYECTADO,INGRESO
71,P-1729,ESTACION CENTRAL,0.00,6,2023,PROYECTADO,MATERIALES
18,P-1670,MK SEGURIDAS,0.00,6,2023,PROYECTADO,INGRESO
20,P-1672,SANTA CLARA,0.00,6,2023,PROYECTADO,MANO DE OBRA


#### 3.4.6 CAMBIO DE VALORES EN COLUMNA TIPO_CUENTA

In [674]:
# Cambiar los valores de la columna TIPO_CUENTA
df_proyectado4['TIPO_CUENTA'] = df_proyectado4['TIPO_CUENTA'].replace({'MATERIALES': '2. COSTO MATERIALES', 'MANO DE OBRA': '3. COSTO MANO DE OBRA', 'INGRESO': '1. INGRESO'})

# Mostrar el DataFrame con los valores cambiados
print("Dimensiones solo df_proyectado4 después de cambios",df_proyectado4.shape)
df_proyectado4.sample(10)

Dimensiones solo df_proyectado4 después de cambios (219, 7)


,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
8,P-1645,LOS TRES ANTONIO,0,6,2023,PROYECTADO,2. COSTO MATERIALES
6,P-1638,COLON,0,6,2023,PROYECTADO,3. COSTO MANO DE OBRA
13,P-1661,SAN VICENTE,0,6,2023,PROYECTADO,2. COSTO MATERIALES
17,P-1669,DEPARTAMENTAL II,0,6,2023,PROYECTADO,3. COSTO MANO DE OBRA
23,P-1676,PORTAL LO CAMPINO,0,6,2023,PROYECTADO,3. COSTO MANO DE OBRA
15,P-1667,ECO PACILLA,0,6,2023,PROYECTADO,3. COSTO MANO DE OBRA
53,P-1709,CC PASEO SAN PEDRO,0,6,2023,PROYECTADO,2. COSTO MATERIALES
35,P-1691,ROSARIO ROSALES,0,6,2023,PROYECTADO,2. COSTO MATERIALES
43,P-1699,VILLA PANAMERICANA LOTE B,-22000000,6,2023,PROYECTADO,2. COSTO MATERIALES
14,P-1663,OFICINAS EQUIFAX,0,6,2023,PROYECTADO,2. COSTO MATERIALES


#### 3.4.6-2 CAMBIO DE VALORES EN COLUMNA TIPO_CUENTA 2

In [675]:
# Cambiar los valores de la columna TIPO_CUENTA
df_proyectado4_1M['TIPO_CUENTA'] = df_proyectado4_1M['TIPO_CUENTA'].replace({'MATERIALES': '2. COSTO MATERIALES', 'MANO DE OBRA': '3. COSTO MANO DE OBRA', 'INGRESO': '1. INGRESO'})

# Mostrar el DataFrame con los valores cambiados
print("Dimensiones solo df_proyectado4 después de cambios",df_proyectado4_1M.shape)
df_proyectado4_1M.sample(10)

Dimensiones solo df_proyectado4 después de cambios (222, 7)


,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
35,P-1691,ROSARIO ROSALES,0.00,6,2023,PROYECTADO,2. COSTO MATERIALES
44,P-1700,ORIZON,0.00,6,2023,PROYECTADO,2. COSTO MATERIALES
39,P-1695,ESTORIL,3000000.00,6,2023,PROYECTADO,2. COSTO MATERIALES
36,P-1692,JOSE PEREZ COTAPO,6800000.00,6,2023,PROYECTADO,1. INGRESO
54,P-1712,CC VIÑA OUTLET PARK,12500000.00,6,2023,PROYECTADO,1. INGRESO
42,P-1698,PCI ANDES,0.00,6,2023,PROYECTADO,3. COSTO MANO DE OBRA
56,P-1714,DEPARTAMENTAL II,0.00,6,2023,PROYECTADO,1. INGRESO
18,P-1670,MK SEGURIDAS,0.00,6,2023,PROYECTADO,3. COSTO MANO DE OBRA
32,P-1688,SUIZA,0.00,6,2023,PROYECTADO,3. COSTO MANO DE OBRA
61,P-1719,METALPARK,0.00,6,2023,PROYECTADO,2. COSTO MATERIALES


In [676]:
df_proyectado4.to_csv('archivos_respaldo/df_proyectado4.csv', mode='a')
df_proyectado4_1M.to_csv('archivos_respaldo/df_proyectado4_1M.csv', mode='a')

### 3.5 TRATAMIENTO LIBRO MAYOR AÑOS ANTERIORES

#### 3.5.1 - REVISIÓN DE LOS DATOS

In [677]:
df_libro_mayor_old.shape

(17368, 13)

In [678]:
df_libro_mayor_old.describe()

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario
count,17368,17368,17368,17368,17368,17368,17368,17368,17368,17368,17368,17368,17368
unique,10,2,12,531,5,1312,3453,134,1500,5119,8462,1714,2
top,4101010-000,2022,9,12/31/22 0:00,CIN,1252,REMUNERACIONES DIRECTAS PROYECTO,,76525290-3,,0,0,
freq,12133,9797,1775,252,11773,251,1359,699,406,1958,3199,15265,17359


#### 3.5.2 - Estandarización campo "Debito" y Crédito

In [679]:
# Se crean nuevos campos para trabajar "Debito" y "Credito".

df_libro_mayor_old['Debito_2'] = pd.to_numeric(df_libro_mayor_old['Debito'], errors='coerce')
df_libro_mayor_old['Credito_2'] = pd.to_numeric(df_libro_mayor_old['Credito'], errors='coerce')

# Se reemplaza el valor de los NoNe y Nan 

df_libro_mayor_old['Credito_2'] = df_libro_mayor_old['Credito_2'].fillna(0)

df_libro_mayor_old.head(5)

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2
1,3101030-000,2021,1,1/6/21 0:00,CVT,47,EP N°26 - OBRA: S-0333 EDIFICIO NC 3000,P-1592,76525290-3,10756,0,6481068,,0.00,6481068.00
2,3101030-000,2021,1,1/6/21 0:00,CVT,47,"UF 222,93 x T/C $29.072,21",P-1592,76525290-3,10756,0,0,,0.00,0.00
3,3101030-000,2021,1,1/6/21 0:00,CVT,47,SEGÚN CONTRATO N°8000033317-00010 - HES N°1000...,P-1592,76525290-3,10756,0,0,,0.00,0.00
4,3101030-000,2021,1,1/6/21 0:00,CVT,60,EP N°4 - OBRA: VIVIPRA,P-1658,79606430-7,10763,0,4301576,,0.00,4301576.00
5,3101030-000,2021,1,1/6/21 0:00,CVT,60,"UF 148 x T/C $29.064,70",P-1658,79606430-7,10763,0,0,,0.00,0.00


#### 3.5.3 - Creación campo Codigo_proyecto

In [680]:
# Se crean una nueva columna para manejar el código del proyecto 

df_libro_mayor_old['CODIGO_PROYECTO'] = df_libro_mayor_old['Primer_Analisis']

cols = df_libro_mayor_old.columns.tolist() # Obtiene el nombre de todas las columnas en una lista
cols.insert(0, cols.pop(cols.index('CODIGO_PROYECTO'))) # Extrae la columna 'Codigo_proyecto', la inserta en la posición 0 y actualiza la lista
df_libro_mayor_old = df_libro_mayor_old.reindex(columns=cols) # Reindexa el dataframe con las columnas en el orden actualizado

# Transformación a mayúscula código del proyecto
df_libro_mayor_old['CODIGO_PROYECTO'] = df_libro_mayor_old['CODIGO_PROYECTO'].str.upper()

df_libro_mayor_old.head(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2
1,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,EP N°26 - OBRA: S-0333 EDIFICIO NC 3000,P-1592,76525290-3,10756,0,6481068,,0.00,6481068.00
2,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,"UF 222,93 x T/C $29.072,21",P-1592,76525290-3,10756,0,0,,0.00,0.00
3,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,SEGÚN CONTRATO N°8000033317-00010 - HES N°1000...,P-1592,76525290-3,10756,0,0,,0.00,0.00
4,P-1658,3101030-000,2021,1,1/6/21 0:00,CVT,60,EP N°4 - OBRA: VIVIPRA,P-1658,79606430-7,10763,0,4301576,,0.00,4301576.00
5,P-1658,3101030-000,2021,1,1/6/21 0:00,CVT,60,"UF 148 x T/C $29.064,70",P-1658,79606430-7,10763,0,0,,0.00,0.00


#### 3.5.4 - Creación de campo total

In [681]:
df_libro_mayor_old['TOTAL'] = df_libro_mayor_old['Credito_2'] - df_libro_mayor_old['Debito_2']
df_libro_mayor_old.head(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL
1,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,EP N°26 - OBRA: S-0333 EDIFICIO NC 3000,P-1592,76525290-3,10756,0,6481068,,0.00,6481068.00,6481068.00
2,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,"UF 222,93 x T/C $29.072,21",P-1592,76525290-3,10756,0,0,,0.00,0.00,0.00
3,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,SEGÚN CONTRATO N°8000033317-00010 - HES N°1000...,P-1592,76525290-3,10756,0,0,,0.00,0.00,0.00
4,P-1658,3101030-000,2021,1,1/6/21 0:00,CVT,60,EP N°4 - OBRA: VIVIPRA,P-1658,79606430-7,10763,0,4301576,,0.00,4301576.00,4301576.00
5,P-1658,3101030-000,2021,1,1/6/21 0:00,CVT,60,"UF 148 x T/C $29.064,70",P-1658,79606430-7,10763,0,0,,0.00,0.00,0.00


#### 3.5.5 - Creación campo tipo_cuenta "basado en el campo cta_Contable"

In [682]:
# Función para categorizar la columna 'Cta_Contable'
def categorizar_cta_contable(cta_contable):
    if cta_contable =="3101030-000" or cta_contable == "3401002-000":
        return '1. INGRESO'
    elif cta_contable == "4101010-000":
        return '2. COSTO MATERIALES'
    elif cta_contable == "4150002-000":
        return '3. COSTO MANO DE OBRA'
    elif cta_contable == "4150002-002":
        return '4. COSTO SUPERVISIÓN'
    else:
        return 'OTRA'

# Crear la nueva columna 'Tipo_cuenta' aplicando la función
df_libro_mayor_old['Tipo_cuenta'] = df_libro_mayor_old['Cta_Contable'].apply(categorizar_cta_contable)
df_libro_mayor_old.sample(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta
10507,P-1698,4101010-000,2022,5,5/9/22 0:00,CIN,630,"Union copla red 2 1/2"" x 2"" RAN RAN",P-1698,P08200234,45343,1268,0,,1268.00,0.00,-1268.00,2. COSTO MATERIALES
7920,P-1691,3101030-000,2022,6,6/23/22 0:00,CVT,304,SEGÚN CONTRATO DEL 01-12-2021 - CORRIENTE DEBILES,P-1691,76009867-1,12598,0,0,,0.00,0.00,0.00,1. INGRESO
17093,P-OFICINA,4150002-003,2022,12,12/31/22 0:00,AJC,1252,REMUNERACIONES DIRECTAS PROYECTO POR DISTRIBUIR,P-Oficina,18581936,,48238,0,,48238.00,0.00,-48238.00,OTRA
9895,P-1693,4101010-000,2022,4,4/6/22 0:00,CIN,581,La Gotita Adhesivo instantaneo 2ml,P-1693,I17451876,44954,1672,0,,1672.00,0.00,-1672.00,2. COSTO MATERIALES
3111,P-1670,4101010-000,2021,4,4/7/21 0:00,CIN,519,Modulo SFP UF-MM-1G,P-1670,O19501068,41541,448570,0,,448570.00,0.00,-448570.00,2. COSTO MATERIALES


In [683]:
df_libro_mayor_old['Tipo_cuenta'].unique()

array(['1. INGRESO', '2. COSTO MATERIALES', 'OTRA',
       '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN'], dtype=object)

In [684]:
# Revisión de los datos creados

df_libro_mayor_old_piv = pd.pivot_table(df_libro_mayor_old, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_old_piv = pd.DataFrame(df_libro_mayor_old_piv.to_records())

#df_libro_mayor_piv.head(5)
df_libro_mayor_old_piv.sample(10)

,Tipo_cuenta,Año,Mes,Credito_2,Debito_2,TOTAL
18,1. INGRESO,2022,4,177303564,80863736,96439828
28,2. COSTO MATERIALES,2021,2,394704,41088338,-40693634
110,OTRA,2022,11,0,22892412,-22892412
23,1. INGRESO,2022,9,456650724,178539015,278111709
92,4. COSTO SUPERVISIÓN,2022,6,0,3571420,-3571420
63,3. COSTO MANO DE OBRA,2022,12,3,78874043,-78874040
114,OTRA,2022,4,2870355,14571338,-11700983
7,1. INGRESO,2021,5,113310222,10911630,102398592
24,2. COSTO MATERIALES,2021,1,1101953,60946682,-59844729
111,OTRA,2022,12,0,38619730,-38619730


#### 3.5.6 - Se eliminan números de proyectos que no deben ser parte del análisis

In [685]:
# Se crea una copia del dataframe original debido que se reduciran las dimensiones
df_libro_mayor_old2 = df_libro_mayor_old.copy()

# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor_old2["CODIGO_PROYECTO"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor_old2.shape)


Proyectos disponibles antes de la reducción : 
['P-1592' 'P-1658' 'P-1638' 'P-1615' 'P-1639' 'S-1601' 'P-1644' 'P-1628'
 'P-1652' 'P-1666' 'P-1653' 'P-1654' 'P-1656' 'P-1665' 'P-1664' 'P-1670'
 'P-1629' 'P-1649' 'P-1657' 'P-1661' 'P-1671' 'P-1645' 'P-1626' 'P-1611'
 'P-1642' 'P-1648' 'P-1659' 'P-1617' 'P-1627' 'P-1641' 'P-1660' 'P-1636'
 'P-1625' 'P-1667' 'P-1672' 'P-1637' 'P-1623' 'P-1674' 'P-1668' 'P-1676'
 'P-1646' 'P-1675' 'P-1678' 'P-1643' 'P-1632' 'P-1673' 'P-1669' 'P-1677'
 'P-1681' 'P-1683' 'P-1679' 'P-1680' 'P-1684' 'P-1689' 'P-1686' 'P-1685'
 'P-1690' 'P-1691' 'P-OFICINA' 'P-PROYECTOS' 'P-1663' 'P-1640' 'P-1655'
 'P-1631' 'P-1647' 'P-1610' 'P-1612' 'P-1527' 'P-1590' 'P-1616' 'P-1622'
 'P-1633' '' 'P-1692' 'P-1687' 'P-1693' 'P-1694' 'P-1695' 'P-1698'
 'P-1696' 'P-1688' 'P-1697' 'P-1700' 'P-1701' 'P-1702' 'P-1703' 'P-1709'
 'P-1707' 'P-1704' 'P-1712' 'P-1713' 'P-1708' 'P-1682' 'P-1714' 'P-1706'
 'P-1705' 'P-1699' 'P-1715' 'P-1716' 'P-1718' 'P-1717' 'P-1613'
 'P-ANALISIS PROYE' 

##### 3.5.6.1 - Se crea un subdataframe para COSTOS SUPERVISIÓN (separando lo que tiene P-OFICINA y cta_contable 4150002-002)

In [686]:
# Se crea un dataframe separado para tratar los que poseen proyecto "P-OFICINA" ya que los costos de supervisión solo aplican al total
df_libro_mayor_old2_2 = df_libro_mayor_old2.copy()

# Filtrar el DataFrame
df_libro_mayor_old2_2 = df_libro_mayor_old2_2[(df_libro_mayor_old2_2['CODIGO_PROYECTO'] == 'P-OFICINA') & (df_libro_mayor_old2_2['Cta_Contable'] == '4150002-002')]

# Mostrar el DataFrame filtrado
df_libro_mayor_old2_2


,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta
16836,P-OFICINA,4150002-002,2022,9,9/30/22 0:00,AJC,1325,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,0,1724280,0,,1724280.00,0.00,-1724280.00,4. COSTO SUPERVISIÓN
16837,P-OFICINA,4150002-002,2022,9,9/30/22 0:00,AJC,1325,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,0,2183185,0,,2183185.00,0.00,-2183185.00,4. COSTO SUPERVISIÓN
16838,P-OFICINA,4150002-002,2022,10,10/31/22 0:00,AJC,1220,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2234754,0,,2234754.00,0.00,-2234754.00,4. COSTO SUPERVISIÓN
16839,P-OFICINA,4150002-002,2022,10,10/31/22 0:00,AJC,1220,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2584134,0,,2584134.00,0.00,-2584134.00,4. COSTO SUPERVISIÓN
16840,P-OFICINA,4150002-002,2022,11,11/30/22 0:00,AJC,1385,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2369840,0,,2369840.00,0.00,-2369840.00,4. COSTO SUPERVISIÓN
16841,P-OFICINA,4150002-002,2022,11,11/30/22 0:00,AJC,1385,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2719503,0,,2719503.00,0.00,-2719503.00,4. COSTO SUPERVISIÓN
16842,P-OFICINA,4150002-002,2022,12,12/31/22 0:00,AJC,1252,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2153705,0,,2153705.00,0.00,-2153705.00,4. COSTO SUPERVISIÓN
16843,P-OFICINA,4150002-002,2022,12,12/31/22 0:00,AJC,1252,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2502914,0,,2502914.00,0.00,-2502914.00,4. COSTO SUPERVISIÓN


In [687]:
# Elimina los registros NaN en la columna Codigo_proyecto
df_libro_mayor_old2 = df_libro_mayor_old2.dropna(subset=['CODIGO_PROYECTO'])

# Define una lista de valores a eliminar
valores_a_eliminar = ['P-PROYECTOS', 'P-SERVICIOS','P-OFICINA', 'P-ANALISIS PROYE', '13056802-5', '16411600-K', '18800250-1','P-MANTENCIÓN','18800250','13056802','16411600']

# Crea una máscara booleana que identifica las filas que contienen los valores a eliminar
mask = df_libro_mayor_old2['CODIGO_PROYECTO'].isin(valores_a_eliminar)

# Crea un nuevo dataframe sin las filas que contienen los valores a eliminar
df_libro_mayor_old2 = df_libro_mayor_old2[~mask]


In [688]:
# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor_old.shape)
print(df_libro_mayor_old["CODIGO_PROYECTO"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor_old2.shape)
print(df_libro_mayor_old2["CODIGO_PROYECTO"].unique())


Proyectos disponibles antes de la reducción : 
(17368, 18)
['P-1592' 'P-1658' 'P-1638' 'P-1615' 'P-1639' 'S-1601' 'P-1644' 'P-1628'
 'P-1652' 'P-1666' 'P-1653' 'P-1654' 'P-1656' 'P-1665' 'P-1664' 'P-1670'
 'P-1629' 'P-1649' 'P-1657' 'P-1661' 'P-1671' 'P-1645' 'P-1626' 'P-1611'
 'P-1642' 'P-1648' 'P-1659' 'P-1617' 'P-1627' 'P-1641' 'P-1660' 'P-1636'
 'P-1625' 'P-1667' 'P-1672' 'P-1637' 'P-1623' 'P-1674' 'P-1668' 'P-1676'
 'P-1646' 'P-1675' 'P-1678' 'P-1643' 'P-1632' 'P-1673' 'P-1669' 'P-1677'
 'P-1681' 'P-1683' 'P-1679' 'P-1680' 'P-1684' 'P-1689' 'P-1686' 'P-1685'
 'P-1690' 'P-1691' 'P-OFICINA' 'P-PROYECTOS' 'P-1663' 'P-1640' 'P-1655'
 'P-1631' 'P-1647' 'P-1610' 'P-1612' 'P-1527' 'P-1590' 'P-1616' 'P-1622'
 'P-1633' '' 'P-1692' 'P-1687' 'P-1693' 'P-1694' 'P-1695' 'P-1698'
 'P-1696' 'P-1688' 'P-1697' 'P-1700' 'P-1701' 'P-1702' 'P-1703' 'P-1709'
 'P-1707' 'P-1704' 'P-1712' 'P-1713' 'P-1708' 'P-1682' 'P-1714' 'P-1706'
 'P-1705' 'P-1699' 'P-1715' 'P-1716' 'P-1718' 'P-1717' 'P-1613'
 'P-ANAL

#### 3.5.7 - Se crea un nuevo campo de fecha basado en la fecha original

In [689]:
df_libro_mayor_old2['Fecha2'] = pd.to_datetime(df_libro_mayor_old2['Año'].astype(str) + '-' + df_libro_mayor_old2['Mes'].astype(str) + '-1')
df_libro_mayor_old2.sample(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
11176,P-1625,4101010-000,2022,6,6/13/22 0:00,CIN,614,"Magnetico Sobrepuesto 7/8"" Cafe",P-1625,S11290200,45751,764,0,,764.00,0.00,-764.00,2. COSTO MATERIALES,2022-06-01
11092,P-1704,4101010-000,2022,6,6/7/22 0:00,CIN,700,Cinta Aisladora 3M Temflex 1500,P-1704,I16430107,45691,1140,0,,1140.00,0.00,-1140.00,2. COSTO MATERIALES,2022-06-01
13418,P-1686,4101010-000,2022,9,9/21/22 0:00,CIN,568,Fibra Optica FO Monomodo 24F G657A2 LSZH,P-1686,C08732816,47246,995000,0,,995000.00,0.00,-995000.00,2. COSTO MATERIALES,2022-09-01
3703,P-1658,4101010-000,2021,5,5/10/21 0:00,CIN,750,Cable Cordon 3x1.5mm,P-1658,T14380049,41997,79100,0,,79100.00,0.00,-79100.00,2. COSTO MATERIALES,2021-05-01
6951,,4150002-001,2021,5,5/31/21 0:00,CRE,980,SIS-100,,10154587,,46879,0,,46879.00,0.00,-46879.00,OTRA,2021-05-01


##### 3.5.7.1 - Se crea un nuevo campo de fecha basado en la fecha original para  COSTOS SUPERVISIÓN

In [690]:
df_libro_mayor_old2_2['Fecha2'] = pd.to_datetime(df_libro_mayor_old2_2['Año'].astype(str) + '-' + df_libro_mayor_old2_2['Mes'].astype(str) + '-1')
df_libro_mayor_old2_2.sample(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
16837,P-OFICINA,4150002-002,2022,9,9/30/22 0:00,AJC,1325,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,0,2183185,0,,2183185.00,0.00,-2183185.00,4. COSTO SUPERVISIÓN,2022-09-01
16839,P-OFICINA,4150002-002,2022,10,10/31/22 0:00,AJC,1220,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2584134,0,,2584134.00,0.00,-2584134.00,4. COSTO SUPERVISIÓN,2022-10-01
16842,P-OFICINA,4150002-002,2022,12,12/31/22 0:00,AJC,1252,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2153705,0,,2153705.00,0.00,-2153705.00,4. COSTO SUPERVISIÓN,2022-12-01
16840,P-OFICINA,4150002-002,2022,11,11/30/22 0:00,AJC,1385,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2369840,0,,2369840.00,0.00,-2369840.00,4. COSTO SUPERVISIÓN,2022-11-01
16841,P-OFICINA,4150002-002,2022,11,11/30/22 0:00,AJC,1385,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2719503,0,,2719503.00,0.00,-2719503.00,4. COSTO SUPERVISIÓN,2022-11-01


In [691]:
df_libro_mayor_old2_2['Tipo_cuenta'].unique()

array(['4. COSTO SUPERVISIÓN'], dtype=object)

#### 3.5.8 - Se realiza pivote final del dataframe

In [692]:
# Se exploran las columnas:
df_libro_mayor_old2.columns

Index(['CODIGO_PROYECTO', 'Cta_Contable', 'Año', 'Mes', 'Fecha', 'Tipo_Comp',
       'Num_Comp', 'Glosa', 'Primer_Analisis', 'Referencia', 'Num_Referencia',
       'Debito', 'Credito', 'Comentario ', 'Debito_2', 'Credito_2', 'TOTAL',
       'Tipo_cuenta', 'Fecha2'],
      dtype='object', name=0)

In [693]:
# crear pivot table sin margenes

df_libro_mayor_old2_piv = pd.pivot_table(df_libro_mayor_old2, values=['Debito_2','Credito_2','TOTAL'], index=['CODIGO_PROYECTO','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_old2_piv = pd.DataFrame(df_libro_mayor_old2_piv.to_records())

df_libro_mayor_old2_piv.sample(5)

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
34,,OTRA,2022-10-01 00:00:00,2022,10,0,9743100,-9743100
1167,P-1685,1. INGRESO,2022-04-01 00:00:00,2022,4,4072874,0,4072874
387,P-1652,1. INGRESO,2021-05-01 00:00:00,2021,5,1921930,0,1921930
279,P-1644,1. INGRESO,2021-07-01 00:00:00,2021,7,8967789,0,8967789
704,P-1667,2. COSTO MATERIALES,2022-09-01 00:00:00,2022,9,0,9321897,-9321897


In [694]:
# Estas son las dimensiones del dataframe pivoteado:
df_libro_mayor_old2_piv.shape

(1632, 8)

##### 3.5.8.1 - Se realiza pivote final del dataframe paera COSTO SUPERVISIÓN

In [695]:
# crear pivot table sin margenes

df_libro_mayor_old2_2_piv = pd.pivot_table(df_libro_mayor_old2_2, values=['Debito_2','Credito_2','TOTAL'], index=['CODIGO_PROYECTO','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_old2_2_piv = pd.DataFrame(df_libro_mayor_old2_2_piv.to_records())

df_libro_mayor_old2_2_piv = df_libro_mayor_old2_2_piv[df_libro_mayor_old2_2_piv['CODIGO_PROYECTO'] != 'TOTAL']

# Se reemplaza el valor P-OFICINA por el valor TOTAL con lo cual se utilizará para unirlo al dataframe que crea las filas para el cálculo de los totales
df_libro_mayor_old2_2_piv['CODIGO_PROYECTO'] = df_libro_mayor_old2_2_piv['CODIGO_PROYECTO'].replace('P-OFICINA', 'TOTAL')

df_libro_mayor_old2_2_piv.sample()

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
2,TOTAL,4. COSTO SUPERVISIÓN,2022-11-01 00:00:00,2022,11,0,5089343,-5089343


#### 3.5.9 - Se crea subdataframe para proyecto que considera el total con nombre "TOTAL", que se detalla por tipo_cuenta


In [696]:
df_libro_mayor_old2_piv2 = pd.pivot_table(df_libro_mayor_old2_piv, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0)
df_libro_mayor_old2_piv2 = pd.DataFrame(df_libro_mayor_old2_piv2.to_records())

# Agregamos la columna Codigo_proyecto con valor "TOTAL" en la primera columna
df_libro_mayor_old2_piv2.insert(loc=0, column='CODIGO_PROYECTO', value='TOTAL')

# Eliminamos los registros con Tipo_cuenta igual a "TOTAL"
df_libro_mayor_old2_piv2 = df_libro_mayor_old2_piv2[df_libro_mayor_old2_piv2['Tipo_cuenta'] != 'TOTAL']
df_libro_mayor_old2_piv2 = df_libro_mayor_old2_piv2[df_libro_mayor_old2_piv2['Tipo_cuenta'] != '']

df_libro_mayor_old2_piv2.sample(5)

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
24,TOTAL,1. INGRESO,2022-12-01 00:00:00,2022,12,346131023,100849557,245281466
20,TOTAL,1. INGRESO,2022-08-01 00:00:00,2022,8,432805709,235626646,197179063
72,TOTAL,3. COSTO MANO DE OBRA,2022-12-01 00:00:00,2022,12,0,78874043,-78874043
52,TOTAL,3. COSTO MANO DE OBRA,2021-04-01 00:00:00,2021,4,0,3598013,-3598013
4,TOTAL,1. INGRESO,2021-04-01 00:00:00,2021,4,165417555,36206601,129210954


##### 3.5.9.1 - Se anexa el dataframe creado con los totales con el dataframe creado a partir del punto 3.1.6.1 que contiene los costos de supervisión

In [697]:
df_libro_mayor_old2_piv_fin = pd.concat([df_libro_mayor_old2_piv2, df_libro_mayor_old2_2_piv], ignore_index=True)
df_libro_mayor_old2_piv_fin.sample(10)

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
60,TOTAL,3. COSTO MANO DE OBRA,2022-01-01 00:00:00,2022,1,0,21186972,-21186972
41,TOTAL,2. COSTO MATERIALES,2022-06-01 00:00:00,2022,6,2998969,71132401,-68133432
54,TOTAL,3. COSTO MANO DE OBRA,2021-07-01 00:00:00,2021,7,0,220289,-220289
55,TOTAL,3. COSTO MANO DE OBRA,2021-08-01 00:00:00,2021,8,0,1844654,-1844654
20,TOTAL,1. INGRESO,2022-09-01 00:00:00,2022,9,456650724,178539015,278111709
43,TOTAL,2. COSTO MATERIALES,2022-08-01 00:00:00,2022,8,988958,115907457,-114918499
68,TOTAL,3. COSTO MANO DE OBRA,2022-09-01 00:00:00,2022,9,0,74019234,-74019234
59,TOTAL,3. COSTO MANO DE OBRA,2021-12-01 00:00:00,2021,12,0,2840209,-2840209
99,TOTAL,OTRA,2022-04-01 00:00:00,2022,4,2037696,3543599,-1505903
15,TOTAL,1. INGRESO,2022-04-01 00:00:00,2022,4,177303564,80863736,96439828


#### 3.5.10 - Se concatenan los 2 dataframes resultantes

In [698]:
print("=======================================================================")
print("Dimensiones del dataframe antes de la concatenación : ")
print(df_libro_mayor_old2_piv_fin.shape)

Dimensiones del dataframe antes de la concatenación : 
(109, 8)


In [699]:
df_libro_mayor_old3 = pd.concat([df_libro_mayor_old2_piv, df_libro_mayor_old2_piv_fin])
df_libro_mayor_old3.sort_values(by=['CODIGO_PROYECTO']).sample(5)
print(df_libro_mayor_old3.shape)

(1741, 8)


#### 3.5.11 - Estandarización de campos 

In [700]:
# Se estandarizan los campos con sus nombres y formatos, para un mayor entendimiento.

df_libro_mayor_old3 = df_libro_mayor_old3.rename(columns={
    'Codigo_proyecto': 'CODIGO_PROYECTO',
    'Tipo_cuenta': 'TIPO_CUENTA',
    'Fecha2': 'FECHA',
    'Año': 'AÑO',
    'Mes': 'MES',
    'Credito_2': 'CREDITO',
    'Debito_2': 'DEBITO'
})

df_libro_mayor_old3['AÑO'] = pd.to_numeric(df_libro_mayor_old3['AÑO'], downcast='integer')
df_libro_mayor_old3['MES'] = pd.to_numeric(df_libro_mayor_old3['MES'], downcast='integer')

df_libro_mayor_old3.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL
665,P-1666,3. COSTO MANO DE OBRA,2021-03-01 00:00:00,2021.00,3.00,0,0,0
1521,P-1703,3. COSTO MANO DE OBRA,2022-11-01 00:00:00,2022.00,11.00,0,385354,-385354
194,P-1638,2. COSTO MATERIALES,2021-03-01 00:00:00,2021.00,3.00,0,195338,-195338
1366,P-1693,3. COSTO MANO DE OBRA,2022-06-01 00:00:00,2022.00,6.00,0,938189,-938189
437,P-1653,2. COSTO MATERIALES,2021-06-01 00:00:00,2021.00,6.00,100132,725928,-625796
1067,P-1678,2. COSTO MATERIALES,2022-02-01 00:00:00,2022.00,2.00,0,114084,-114084
1273,P-1690,2. COSTO MATERIALES,2022-05-01 00:00:00,2022.00,5.00,59100,2753224,-2694124
1396,P-1696,1. INGRESO,2022-08-01 00:00:00,2022.00,8.00,6509010,0,6509010
578,P-1661,2. COSTO MATERIALES,2022-01-01 00:00:00,2022.00,1.00,71140,69490,1650
1534,P-1704,2. COSTO MATERIALES,2022-10-01 00:00:00,2022.00,10.00,23459,9875116,-9851657


#### 3.5.14 Creación de columna MARGEN

In [701]:
df_libro_mayor_old3_2 = df_libro_mayor_old3.copy()

#df_libro_mayor4_2['TOTAL'] = pd.to_numeric(df_libro_mayor4_2['AÑO'], downcast='float')

df_margen = df_libro_mayor_old3_2.groupby(['CODIGO_PROYECTO', 'MES']).apply(lambda x: pd.Series({
    'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['INGRESO','COSTO SUPERVISIÓN', 'COSTO MANO DE OBRA', 'COSTO MATERIALES']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == 'INGRESO', 'TOTAL'].sum()
}))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)

/var/folders/26/wv7th2z157sgk1cmh3h23d440000gn/T/ipykernel_2049/3531166699.py:6: RuntimeWarning: invalid value encountered in scalar divide
  'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['INGRESO','COSTO SUPERVISIÓN', 'COSTO MANO DE OBRA', 'COSTO MATERIALES']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == 'INGRESO', 'TOTAL'].sum()


,CODIGO_PROYECTO,MES,MARGEN
301,P-1664,11.00,0.00
486,P-1685,7.00,0.00
566,P-1693,11.00,0.00
693,TOTAL,12.00,0.00
607,P-1700,6.00,0.00
316,P-1666,11.00,0.00
251,P-1658,8.00,0.00
105,P-1637,6.00,0.00
453,P-1681,2.00,0.00
261,P-1660,3.00,0.00


In [702]:
print("=======================================================================")
print("Dimensiones del dataframe antes del merge : ")
print(df_libro_mayor_old3.shape)
print("=======================================================================")
print("Dimensiones del dataframe antes del merge : ")
print(df_margen.shape)

Dimensiones del dataframe antes del merge : 
(1741, 8)
Dimensiones del dataframe antes del merge : 
(694, 3)


In [703]:
df_libro_mayor_old3 = pd.merge(df_libro_mayor_old3, df_margen, on=['CODIGO_PROYECTO', 'MES'], how='left')
#EJEMPLO
#df_libro_mayor5[df_libro_mayor5["CODIGO_PROYECTO"]=="P-1706"]
df_libro_mayor_old3[df_libro_mayor_old3["CODIGO_PROYECTO"]=="TOTAL"]

# Se estandariza el valor TOTAL a 1.TOTAL para que pueda ser seleccionado desde los filtros del Dashboard.
df_libro_mayor_old3['CODIGO_PROYECTO'] = df_libro_mayor_old3['CODIGO_PROYECTO'].apply(lambda x: '1. TOTAL' if x == 'TOTAL' else x)

In [704]:
print("=======================================================================")
print("Dimensiones del dataframe después de la concatenación : ")
print(df_libro_mayor_old3.shape)

Dimensiones del dataframe después de la concatenación : 
(1741, 9)


In [705]:
df_libro_mayor_old3.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1209,P-1686,3. COSTO MANO DE OBRA,2021-12-01 00:00:00,2021.00,12.00,0,0,0,0.00
1182,P-1685,3. COSTO MANO DE OBRA,2021-12-01 00:00:00,2021.00,12.00,0,0,0,0.00
1324,P-1692,1. INGRESO,2022-07-01 00:00:00,2022.00,7.00,2054508,0,2054508,0.00
1047,P-1677,3. COSTO MANO DE OBRA,2022-05-01 00:00:00,2022.00,5.00,0,712217,-712217,0.00
1699,1. TOTAL,3. COSTO MANO DE OBRA,2022-08-01 00:00:00,2022.00,8.00,0,66998352,-66998352,0.00
167,P-1636,2. COSTO MATERIALES,2021-02-01 00:00:00,2021.00,2.00,121215,122250,-1035,0.00
1698,1. TOTAL,3. COSTO MANO DE OBRA,2022-07-01 00:00:00,2022.00,7.00,0,63122058,-63122058,0.00
352,P-1648,2. COSTO MATERIALES,2022-06-01 00:00:00,2022.00,6.00,6685,0,6685,0.00
1592,P-1714,1. INGRESO,2022-11-01 00:00:00,2022.00,11.00,5612327,0,5612327,0.00
742,P-1668,2. COSTO MATERIALES,2021-07-01 00:00:00,2021.00,7.00,0,168203,-168203,0.00


In [706]:
df_libro_mayor_old3.to_csv('archivos_respaldo/df_libro_mayor_old3.csv', mode='a')

### 3.6 TRATAMIENTO DE "PLANILLA PRESUPUESTO"

In [707]:
# Se crea una copia del dataframe llamado "presupuesto"
df_presupuesto2 = df_presupuesto.copy()

In [708]:
# Eliminar primera fila completa
df_presupuesto2 = df_presupuesto2.iloc[1:]

# Filtrar y mantener solo las columnas deseadas
df_presupuesto2 = df_presupuesto2[['PROYECTO', 'MONTO EN $']]

df_presupuesto2 = df_presupuesto2.rename(columns={'PROYECTO': 'CODIGO_PROYECTO', 'MONTO EN $': 'VALOR'})

df_presupuesto2.sample(10)

,CODIGO_PROYECTO,VALOR
7,P-1656,36463980.00
43,P-1699,406744605.61
62,P-1720,10690929.98
36,P-1692,91720000.00
14,P-1670,202553946.00
56,P-1714,61454685.57
9,P-1661,48586986.00
24,P-1680,206761360.91
49,P-1705,65200940.03
30,P-1686,105081270.13


In [709]:
# Se agregan nuevas columnas tipo de cuenta y Origen.

df_presupuesto2["TIPO_CUENTA"]= "1. INGRESO"
df_presupuesto2["ORIGEN"]= "PLANILLA PRESUPUESTO"

df_presupuesto2.sample(10)

,CODIGO_PROYECTO,VALOR,TIPO_CUENTA,ORIGEN
45,P-1701,74623683.22,1. INGRESO,PLANILLA PRESUPUESTO
54,P-1712,215321946.83,1. INGRESO,PLANILLA PRESUPUESTO
6,P-1655,51593018.53,1. INGRESO,PLANILLA PRESUPUESTO
56,P-1714,61454685.57,1. INGRESO,PLANILLA PRESUPUESTO
15,P-1671,125912115.00,1. INGRESO,PLANILLA PRESUPUESTO
48,P-1704,1058718831.87,1. INGRESO,PLANILLA PRESUPUESTO
9,P-1661,48586986.00,1. INGRESO,PLANILLA PRESUPUESTO
18,S-1601,16486000.00,1. INGRESO,PLANILLA PRESUPUESTO
16,P-1672,134443080.00,1. INGRESO,PLANILLA PRESUPUESTO
24,P-1680,206761360.91,1. INGRESO,PLANILLA PRESUPUESTO


### 3.7 TRATAMIENTO DE "TABLA PRESUPUESTO"

In [710]:
# Se crea una copia del dataframe para realizar el tratamiento.

df_tabla_presupuesto2 = df_tabla_prespuesto.copy()
print("DIMENSIONES DATAFRAME df_tabla_prespuesto2", df_tabla_presupuesto2.shape)
df_tabla_presupuesto2.sample(5)

DIMENSIONES DATAFRAME df_tabla_prespuesto2 (13826, 7)


,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL
12304,P-1718,1001,Central de Incendio-Rayos Detector Humo,UNIDA,3,589000,1767000
8255,P-1683,1073,"Soportes y Antisismicos-Perno U 8"""" """,UNIDA,2,1472,2944
7481,P-1674,1122,Gastos Generales-Examen PCR,Gl,1,400370,400370
12761,P-1720,1076,Telecomunicaciones -Derivadores Alcad FD-419,,2,5000,10000
9111,P-1691,1051,Portón-Baliza Giratoria,,3,30000,90000


In [711]:
# Se estandarizan los tipos de campo y se reemplazan valores NaN

df_tabla_presupuesto2['COSTO_UNITARIO'] = pd.to_numeric(df_tabla_presupuesto2['COSTO_UNITARIO'], errors='coerce')
df_tabla_presupuesto2['COSTO_TOTAL'] = pd.to_numeric(df_tabla_presupuesto2['COSTO_TOTAL'], errors='coerce')

# Se reemplaza el valor de los NoNe y Nan 

df_tabla_presupuesto2['COSTO_UNITARIO'] = df_tabla_presupuesto2['COSTO_UNITARIO'].fillna(0)
df_tabla_presupuesto2['COSTO_TOTAL'] = df_tabla_presupuesto2['COSTO_TOTAL'].fillna(0)

print("DIMENSIONES DATAFRAME df_tabla_prespuesto2", df_tabla_presupuesto2.shape)
df_tabla_presupuesto2.head(5)

DIMENSIONES DATAFRAME df_tabla_prespuesto2 (13826, 7)


,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL
0,P-1602,1001,DETECCIÓN-Central de detección Hochiki FNP112...,,1,428490.00,428490.00
1,P-1602,1002,DETECCIÓN-Sensor Humo Hochiki ALN-C,,49,20866.00,1022434.00
2,P-1602,1003,DETECCIÓN-Sensor de temperatura Hochiki ATJ-EA,,4,23846.00,95384.00
3,P-1602,1004,DETECCIÓN-Base Sensores Hochiki HSB-NSA-6,,53,2608.00,138224.00
4,P-1602,1005,DETECCIÓN-Palanca Manual Hochiki DCP-AMS,,13,27200.00,353600.00


In [712]:
#### Creación de campo tipo_costo basado en el valor del a columna "PRODUCTO"

def clasificar_tipo_costo(producto):
    if   producto.upper().startswith("MANO DE OBRA"):
        return "3. COSTO MANO DE OBRA"
    elif producto.upper().startswith("SUPER"):
        return "4. COSTO SUPERVISIÓN"
    else:
        return "2. COSTO MATERIALES"

df_tabla_presupuesto2['TIPO_COSTO'] = df_tabla_presupuesto2['PRODUCTO'].apply(clasificar_tipo_costo)

df_tabla_presupuesto2.head(5)

,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL,TIPO_COSTO
0,P-1602,1001,DETECCIÓN-Central de detección Hochiki FNP112...,,1,428490.00,428490.00,2. COSTO MATERIALES
1,P-1602,1002,DETECCIÓN-Sensor Humo Hochiki ALN-C,,49,20866.00,1022434.00,2. COSTO MATERIALES
2,P-1602,1003,DETECCIÓN-Sensor de temperatura Hochiki ATJ-EA,,4,23846.00,95384.00,2. COSTO MATERIALES
3,P-1602,1004,DETECCIÓN-Base Sensores Hochiki HSB-NSA-6,,53,2608.00,138224.00,2. COSTO MATERIALES
4,P-1602,1005,DETECCIÓN-Palanca Manual Hochiki DCP-AMS,,13,27200.00,353600.00,2. COSTO MATERIALES


In [713]:
# crear pivot table sin margenes

df_tabla_presupuesto3 = pd.pivot_table(df_tabla_presupuesto2, values=['COSTO_TOTAL'], index=['CODIGO_PROYECTO','TIPO_COSTO'], aggfunc='sum', fill_value=0)

df_tabla_presupuesto3 = pd.DataFrame(df_tabla_presupuesto3.to_records())
df_tabla_presupuesto3.sample(10)


,CODIGO_PROYECTO,TIPO_COSTO,COSTO_TOTAL
300,P-1704,2. COSTO MATERIALES,387816472.00
271,P-1692,4. COSTO SUPERVISIÓN,5768030.00
137,P-1647,4. COSTO SUPERVISIÓN,307920.00
86,P-1630,4. COSTO SUPERVISIÓN,2804904.00
387,S-1601,4. COSTO SUPERVISIÓN,819548.00
103,P-1636,3. COSTO MANO DE OBRA,6643720.00
193,P-1666,3. COSTO MANO DE OBRA,8652090.00
239,P-1681,4. COSTO SUPERVISIÓN,532619.00
268,P-1691,4. COSTO SUPERVISIÓN,2099697.00
63,P-1623,2. COSTO MATERIALES,15611718.00


In [714]:

# Se crea columna VALOR y se considera el valor en negativo.
df_tabla_presupuesto3['TOTAL'] = df_tabla_presupuesto3['COSTO_TOTAL'].apply(lambda x: x * -1 if x != 0 else 0)

# Se crea la columna de ORIGEN del archivo
df_tabla_presupuesto3["ORIGEN"] = "TABLA_PRESUPUESTO"

print("DIMENSIÓN DE LA TABLA ES", df_tabla_presupuesto3.shape)
df_tabla_presupuesto3.sample(5)

DIMENSIÓN DE LA TABLA ES (388, 5)


,CODIGO_PROYECTO,TIPO_COSTO,COSTO_TOTAL,TOTAL,ORIGEN
44,P-1616,4. COSTO SUPERVISIÓN,1234269.00,-1234269.00,TABLA_PRESUPUESTO
63,P-1623,2. COSTO MATERIALES,15611718.00,-15611718.00,TABLA_PRESUPUESTO
139,P-1648,3. COSTO MANO DE OBRA,33676709.00,-33676709.00,TABLA_PRESUPUESTO
62,P-1622,4. COSTO SUPERVISIÓN,642985.00,-642985.00,TABLA_PRESUPUESTO
30,P-1612,2. COSTO MATERIALES,30609079.00,-30609079.00,TABLA_PRESUPUESTO


In [715]:
# INSPECCIÓN VISUAL
df_tabla_presupuesto_testing = pd.pivot_table(df_tabla_presupuesto3, values=['TOTAL'], index=['TIPO_COSTO'], aggfunc='sum', fill_value=0)
df_tabla_presupuesto_testing

,TOTAL
TIPO_COSTO,
2. COSTO MATERIALES,-5745141472.04
3. COSTO MANO DE OBRA,-1482885459.00
4. COSTO SUPERVISIÓN,-311477854.00


In [716]:
df_tabla_presupuesto3.to_csv('archivos_respaldo/df_tabla_presupuesto3.csv', mode='a')

## 4. - MERGE Y TRATAMIENTO DE DATAFRAMES 

### 4.1 - MERGE PARA -> CUADRO FINANCIERO PROYECTOS 
Merge entre Dataframes: df_libro_mayor5 y df_cod_proyecto

In [739]:
print("Dimensiones antes del merge:")
print(df_libro_mayor5.shape)

df_libro_mayor6 = pd.merge(df_libro_mayor5, df_cod_proyecto, on=['CODIGO_PROYECTO'], how='left')

print("Dimensiones después del merge:")
print(df_libro_mayor6.shape)

Dimensiones antes del merge:
(2515, 9)
Dimensiones después del merge:
(2515, 11)


In [740]:
# Reemplazar los valores NaN por "TOTAL" cuando el CODIGO_PROYECTO sea "TOTAL"

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
df_libro_mayor6[cols_to_replace] = df_libro_mayor6[cols_to_replace].replace('TOTAL', '1. TOTAL')

# Quizás el código anterior sea redundante.
mask = df_libro_mayor6['CODIGO_PROYECTO'] == '1. TOTAL'
df_libro_mayor6.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

# Se elimina la columna MERGE la que es innecesaria tener hasta ahora
df_libro_mayor6 = df_libro_mayor6.drop('MARGEN', axis=1)

df_libro_mayor6[df_libro_mayor6["CODIGO_PROYECTO"]== "1. TOTAL"].head(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
502,1. TOTAL,,,NaN,NaN,2201473074,1665348788,536124286,1. TOTAL,1. TOTAL
503,1. TOTAL,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,72482994,-72482994,1. TOTAL,1. TOTAL
504,1. TOTAL,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,82645544,-82645544,1. TOTAL,1. TOTAL
505,1. TOTAL,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,96760244,-96760244,1. TOTAL,1. TOTAL
506,1. TOTAL,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,111885468,-111885468,1. TOTAL,1. TOTAL


#### 4.1.1 - AGREGACIÓN DE NUEVA COLUMNA DE TOTAL PARA AGREGAR EN GRÁFICO.
Se agrega nueva columna de total pero considerando el valor de los tipos de costo con valores positivos.

In [741]:
# Crear la nueva columna 'TOTAL_2' con los valores de 'TOTAL' positivos
df_libro_mayor6['TOTAL_2'] = np.where(df_libro_mayor6['TIPO_CUENTA'] != 'INGRESO', np.abs(df_libro_mayor6['TOTAL']), df_libro_mayor6['TOTAL'])
df_libro_mayor6.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_2
888,P-1648,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,0,0,0,Etapa 2 CD Peralillo TW,Inmobiliaria y Rentas Logística S.A.,0
1653,P-1691,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,0,0,Rosario Rosales,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,0
1758,P-1695,COSTO SUPERVISIÓN,2023-02-01 00:00:00,2023.00,2.00,0,0,0,Edificio Estoril,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,0
1580,P-1688,COSTO MATERIALES,2023-04-01 00:00:00,2023.00,4.00,0,0,0,Suiza,Moller y Perez - Cotapos Construcciones Indust...,0
1466,P-1683,COSTO SUPERVISIÓN,2023-01-01 00:00:00,2023.00,1.00,0,0,0,CD BP-A05,EBCO S.A.,0
1894,P-1701,COSTO SUPERVISIÓN,2023-04-01 00:00:00,2023.00,4.00,0,0,0,Edificio Zenteno,CONSTRUCTORA GREVIA SpA.,0
1374,P-1679,COSTO SUPERVISIÓN,2023-02-01 00:00:00,2023.00,2.00,0,0,0,Juan Sabaj,Constructora Grevia,0
1638,P-1690,COSTO SUPERVISIÓN,2023-02-01 00:00:00,2023.00,2.00,0,0,0,Ongolmo - Home Condominio,CANTAUCO CONSTRUCCION LTDA.,0
1135,P-1667,INGRESO,2023-03-01 00:00:00,2023.00,3.00,0,0,0,Eco Placilla,Constructora ICF SPA,0
687,P-1620,INGRESO,2023-06-01 00:00:00,2023.00,6.00,0,0,0,"OF Apoquindo- Habilitaciones, Integracion y ad...",CONSTRUCTORA INARCO S.A.,0


#### 4.1.2 - AJUSTES DE COLUMNAS 

In [742]:
# Renombrar los valores de la columna CODIGO_PROYECTO
df_libro_mayor6['TIPO_CUENTA'] = df_libro_mayor6['TIPO_CUENTA'].replace({
    'INGRESO': '1. INGRESO',
    'COSTO MATERIALES': '2. COSTO MATERIALES',
    'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA',
    'COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'
})

df_libro_mayor6['CODIGO_PROYECTO'] = df_libro_mayor6['CODIGO_PROYECTO'].replace({
    'TOTAL': '1. TOTAL'})

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_2
1587,P-1688,1. INGRESO,2023-02-01 00:00:00,2023.00,2.00,0,0,0,Suiza,Moller y Perez - Cotapos Construcciones Indust...,0
1422,P-1681,4. COSTO SUPERVISIÓN,2023-02-01 00:00:00,2023.00,2.00,0,0,0,CD Rosen (CCTV Provisorio),Colchones ROSEN SAIC,0
2483,P-1728,1. INGRESO,2023-05-01 00:00:00,2023.00,5.00,0,0,0,NaN,NaN,0
1481,P-1683,3. COSTO MANO DE OBRA,2023-06-01 00:00:00,2023.00,6.00,0,0,0,CD BP-A05,EBCO S.A.,0
1586,P-1688,4. COSTO SUPERVISIÓN,2023-01-01 00:00:00,2023.00,1.00,0,0,0,Suiza,Moller y Perez - Cotapos Construcciones Indust...,0
1365,P-1679,3. COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,0,0,Juan Sabaj,Constructora Grevia,0
2465,P-1727,3. COSTO MANO DE OBRA,2023-06-01 00:00:00,2023.00,6.00,0,0,0,NaN,NaN,0
534,P-1557,4. COSTO SUPERVISIÓN,2023-02-01 00:00:00,2023.00,2.00,0,0,0,Rosario Rosales,EBCO S.A.,0
1904,P-1701,2. COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,0,0,0,Edificio Zenteno,CONSTRUCTORA GREVIA SpA.,0
1794,P-1696,4. COSTO SUPERVISIÓN,2023-06-01 00:00:00,2023.00,6.00,0,0,0,Santa Victoria - Terracorp,Constructora Terracorp S.A.,0


In [743]:
# Ordenar el DataFrame por la columna CODIGO_PROYECTO
df_libro_mayor6 = df_libro_mayor6.sort_values('CODIGO_PROYECTO')

# Mover la fila con CODIGO_PROYECTO = '1. TOTAL' al primer lugar
df_libro_mayor6 = pd.concat([df_libro_mayor6[df_libro_mayor6['CODIGO_PROYECTO'] == '1. TOTAL'],
                            df_libro_mayor6[df_libro_mayor6['CODIGO_PROYECTO'] != '1. TOTAL']])

df_libro_mayor6['NOMBRE_OBRA'] = df_libro_mayor6['NOMBRE_OBRA'].str.upper()
df_libro_mayor6['NOMBRE_CLIENTE'] = df_libro_mayor6['NOMBRE_CLIENTE'].str.upper()

# Reiniciar los índices del DataFrame
df_libro_mayor6 = df_libro_mayor6.reset_index(drop=True)
df_libro_mayor6.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_2
419,P-1645,2. COSTO MATERIALES,2023-05-01 00:00:00,2023.00,5.00,0,0,0,LOS TRES ANTONIOS,SOCIEDAD CONSTRUCTORA ECHAVARRI HERMANOS LIMITADA,0
1374,P-1692,3. COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,0,0,JOSÉ PEREZ COTAPOS,EBCO S.A.,0
1168,P-1686,2. COSTO MATERIALES,2023-06-01 00:00:00,2023.00,6.00,0,0,0,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,0
989,P-1680,3. COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,0,0,NUEVA UNO,CONSTRUCTORA TECSA S.A.,0
1693,P-1701,3. COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,768063,-768063,EDIFICIO ZENTENO,CONSTRUCTORA GREVIA SPA.,768063


#### 4.1.3 - CREACIÓN DE COLUMNA MARGEN

In [744]:
df_margen = df_libro_mayor6.groupby(['CODIGO_PROYECTO', 'MES']).apply(lambda x: pd.Series({
    'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA','4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
}))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

#warnings.filterwarnings("ignore", category=FutureWarning, message="Inferring datetime64.*")

# mostrar el DataFrame resultante
df_margen.sample(10)

df_libro_mayor6 = pd.merge(df_libro_mayor6, df_margen, on=['CODIGO_PROYECTO', 'MES'], how='left')

# INSPECCIÓN VISUAL 1
df_libro_mayor6[df_libro_mayor6["CODIGO_PROYECTO"]=="P-1704"]


/var/folders/26/wv7th2z157sgk1cmh3h23d440000gn/T/ipykernel_2049/1823500159.py:2: RuntimeWarning: invalid value encountered in scalar divide
  'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA','4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
/var/folders/26/wv7th2z157sgk1cmh3h23d440000gn/T/ipykernel_2049/1823500159.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA','4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_2,MARGEN
1752,P-1704,4. COSTO SUPERVISIÓN,2023-06-01 00:00:00,2023.00,6.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
1753,P-1704,3. COSTO MANO DE OBRA,2023-06-01 00:00:00,2023.00,6.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
1754,P-1704,2. COSTO MATERIALES,2023-06-01 00:00:00,2023.00,6.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
1755,P-1704,1. INGRESO,2023-06-01 00:00:00,2023.00,6.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
1756,P-1704,4. COSTO SUPERVISIÓN,2023-05-01 00:00:00,2023.00,5.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.82
1757,P-1704,3. COSTO MANO DE OBRA,2023-05-01 00:00:00,2023.00,5.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.82
1758,P-1704,2. COSTO MATERIALES,2023-05-01 00:00:00,2023.00,5.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.82
1759,P-1704,1. INGRESO,2023-05-01 00:00:00,2023.00,5.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.82
1760,P-1704,4. COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023.00,3.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.04
1761,P-1704,2. COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.04


In [749]:
df_libro_mayor6.to_csv('archivos_respaldo/df_libro_mayor6.csv', mode='a')

### 4.2 - MERGE PARA -> YTD AÑO EN CURSO HASTA MES CERRADO
MERGE ENTRE df_libro_mayor_ytd_closed_piv y df_cod_proyecto

In [751]:
print("Dimensiones antes del merge:")
df_libro_mayor_ytd_closed_piv.shape

Dimensiones antes del merge:


(332, 3)

In [772]:
df_ytd_cerrado = df_libro_mayor_ytd_closed_piv.copy()

In [754]:
# Se realiza merge con Nombre de Obra y Cliente.

print("Dimensiones mes_en_curso antes de cambios: ", df_ytd_cerrado.shape)
df_ytd_cerrado = df_ytd_cerrado.merge(df_cod_proyecto, on='CODIGO_PROYECTO', how='left')
print("Dimensiones mes_en_curso2 antes de cambios: ", df_ytd_cerrado.shape)
df_ytd_cerrado.sample(10)

Dimensiones mes_en_curso antes de cambios:  (332, 3)
Dimensiones mes_en_curso2 antes de cambios:  (332, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
30,P-1620,COSTO SUPERVISIÓN,0,"OF Apoquindo- Habilitaciones, Integracion y ad...",CONSTRUCTORA INARCO S.A.
59,P-1644,INGRESO,0,Mirador de San Cristobal,EBCO S.A.
260,P-1709,COSTO MANO DE OBRA,-9642970,Centro Comercial Paseo San Pedro,EBCO S.A.
201,P-1693,COSTO MATERIALES,103024,Volta V0800,ECOBIO SPA
110,P-1668,COSTO SUPERVISIÓN,0,Quillayes Plaza,E.MOLINA MOREL CONSTRUCTORA S.A.
141,P-1677,COSTO MATERIALES,-1379557,Galvarino Gallardo,Moller y Perez - Cotapos Construcciones Indust...
24,P-1612,COSTO MANO DE OBRA,0,Barrio Parque Santiago 7,CONSTRUCTORA ALTURAS DEL SUR S.A
61,P-1645,COSTO MATERIALES,19980,Los tres Antonios,Sociedad Constructora Echavarri Hermanos Limitada
68,P-1649,COSTO MANO DE OBRA,0,Inacap Vitacura E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.
187,P-1689,INGRESO,0,Casino Buenaventura,EBCO S.A.


In [755]:
# Se reemplazan nombres de TIPO_CUENTA

df_ytd_cerrado['TIPO_CUENTA'] = df_ytd_cerrado['TIPO_CUENTA'].replace({
    'INGRESO': '1. INGRESO',
    'COSTO MATERIALES': '2. COSTO MATERIALES',
    'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA',
    'COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'
})
df_ytd_cerrado.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
40,P-1639,3. COSTO MANO DE OBRA,0,Lyon 2546,Moller y Perez - Cotapos Construcciones Indust...
106,P-1667,4. COSTO SUPERVISIÓN,0,Eco Placilla,Constructora ICF SPA
39,P-1632,1. INGRESO,0,Chorrillos Clorox,Constructora Suksa S.A.
199,P-1692,1. INGRESO,46902525,José Perez Cotapos,EBCO S.A.
285,P-1717,2. COSTO MATERIALES,0,Eco Quilín I,Constructora ICF SPA
31,P-1620,1. INGRESO,0,"OF Apoquindo- Habilitaciones, Integracion y ad...",CONSTRUCTORA INARCO S.A.
281,P-1716,2. COSTO MATERIALES,-24647495,CD Astara,Astara Logística Chile SpA.
317,P-1725,2. COSTO MATERIALES,-10888672,Lo Cañas,Constructora Brisas de Batuco
117,P-1670,2. COSTO MATERIALES,6945,MK II Seguridad y Redes,Comercial K SpA.
232,P-1701,3. COSTO MANO DE OBRA,-891528,Edificio Zenteno,CONSTRUCTORA GREVIA SpA.


In [756]:
# INSPECCIÓN VISUAL 1 
df_ytd_cerrado[df_ytd_cerrado["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-363774250,NaN,NaN
1,1. TOTAL,2. COSTO MATERIALES,-604151999,NaN,NaN
2,1. TOTAL,4. COSTO SUPERVISIÓN,-17923405,NaN,NaN
3,1. TOTAL,1. INGRESO,1520471350,NaN,NaN


In [757]:
#Se renombra "TOTAL" a "1. TOTAL" para que se pueda visualizar en el dashboard.

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
df_ytd_cerrado[cols_to_replace] = df_ytd_cerrado[cols_to_replace].replace('TOTAL', '1. TOTAL')
mask = df_ytd_cerrado['CODIGO_PROYECTO'] == '1. TOTAL'
df_ytd_cerrado.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'
df_ytd_cerrado.sample(4)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
244,P-1705,3. COSTO MANO DE OBRA,-8357637,Los Clarines,CONSTRUCTORA INARCO S.A.
294,P-1719,4. COSTO SUPERVISIÓN,0,CD METALPAR,EBCO S.A.
60,P-1645,3. COSTO MANO DE OBRA,0,Los tres Antonios,Sociedad Constructora Echavarri Hermanos Limitada
149,P-1680,2. COSTO MATERIALES,-38487045,Nueva Uno,Constructora Tecsa S.A.


In [758]:
# INSPECCIÓN VISUAL 1
df_ytd_cerrado[df_ytd_cerrado["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-363774250,1. TOTAL,1. TOTAL
1,1. TOTAL,2. COSTO MATERIALES,-604151999,1. TOTAL,1. TOTAL
2,1. TOTAL,4. COSTO SUPERVISIÓN,-17923405,1. TOTAL,1. TOTAL
3,1. TOTAL,1. INGRESO,1520471350,1. TOTAL,1. TOTAL


#### 4.2.1 - SE REALIZA CRUCE CON "df_libro_mayor5_base2" PARA CREAR DATAFRAME FINAL

In [760]:
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado   : ", df_ytd_cerrado.shape)

df_ytd_cerrado2 = df_ytd_cerrado.filter(["CODIGO_PROYECTO", "TIPO_CUENTA", "TOTAL"])

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado2 : ", df_ytd_cerrado2.shape)

df_ytd_cerrado2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado   :  (332, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado2 :  (332, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
182,P-1620,1. INGRESO,0,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.
145,P-1721,1. INGRESO,0,STRIP CENTER LUIS PASTEUR,INMOBILIARIA LUIS PASTEUR S.A.
263,P-1680,4. COSTO SUPERVISIÓN,0,NUEVA UNO,CONSTRUCTORA TECSA S.A.
269,P-1683,3. COSTO MANO DE OBRA,0,CD BP-A05,EBCO S.A.
303,P-1713,4. COSTO SUPERVISIÓN,0,RESERVA SAN FRANCISCO,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...


In [761]:
# Exploración dataframe base

df_libro_mayor5_base2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
58,P-1686,2. COSTO MATERIALES,0,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.
304,P-1714,4. COSTO SUPERVISIÓN,0,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA
69,P-1690,1. INGRESO,0,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.
166,P-1557,4. COSTO SUPERVISIÓN,0,ROSARIO ROSALES,EBCO S.A.
117,P-1708,2. COSTO MATERIALES,0,ECO ESPAÑA,CONSTRUCTORA ICF SPA


In [762]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

df_ytd_cerrado3 = df_libro_mayor5_base2.merge(df_ytd_cerrado2, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3       : ", df_ytd_cerrado3.shape)

df_ytd_cerrado3.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (332, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3       :  (332, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
92,P-1699,3. COSTO MANO DE OBRA,0,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,-18299646
37,P-1675,1. INGRESO,0,PALENA II,CONSTRUCTORA MYA SPA.,398016
148,P-1723,2. COSTO MATERIALES,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-44856698
227,P-1661,3. COSTO MANO DE OBRA,0,SAN VICENTE DE PAUL,DANACORP S.A,0
150,P-1724,3. COSTO MANO DE OBRA,0,NaN,NaN,-506066


In [763]:
# INSPECCIÓN VISUAL 1

df_ytd_cerrado3[df_ytd_cerrado3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
328,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-17923405
329,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1520471350
330,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-604151999
331,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-363774250


In [764]:
# INSPECCIÓN VISUAL 2

df_ytd_cerrado3[df_ytd_cerrado3["CODIGO_PROYECTO"]=="P-1724"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
150,P-1724,3. COSTO MANO DE OBRA,0,NaN,NaN,-506066
151,P-1724,2. COSTO MATERIALES,0,NaN,NaN,-7710229
152,P-1724,1. INGRESO,0,NaN,NaN,28935273
322,P-1724,4. COSTO SUPERVISIÓN,0,NaN,NaN,0


#### 4.2.2 - SE REALIZA CREACIÓN DE COLUMNA MARGEN

In [765]:
# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ytd_cerrado3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES','3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_margen       : ", df_margen.shape)

# mostrar el DataFrame resultante
df_margen.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_margen       :  (83, 2)


,CODIGO_PROYECTO,MARGEN
5,P-1611,0.00
20,P-1658,0.00
60,P-1704,0.47
80,P-1726,0.43
18,P-1653,0.00
11,P-1641,0.00
13,P-1643,0.00
40,P-1683,0.00
73,P-1719,1.00
25,P-1666,0.00


In [766]:
df_margen[df_margen["CODIGO_PROYECTO"]=="P-1724"]

,CODIGO_PROYECTO,MARGEN
78,P-1724,0.72


In [767]:
# Merge Dataframe para agregar MARGEN

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3   : ", df_ytd_cerrado3.shape)

df_ytd_cerrado3 = df_ytd_cerrado3.merge(df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3 : ", df_ytd_cerrado3.shape)

df_ytd_cerrado3.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3   :  (332, 6)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3 :  (332, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
96,P-1700,2. COSTO MATERIALES,0,CD ORIZON,EBCO S.A.,-7753684,0.28
114,P-1707,2. COSTO MATERIALES,0,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,-8495556,-0.09
113,P-1707,3. COSTO MANO DE OBRA,0,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,-3813841,-0.09
38,P-1676,3. COSTO MANO DE OBRA,0,LO CAMPINO,EBCO S.A.,-1798488,0.00
229,P-1663,1. INGRESO,0,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA,0,0.00


In [768]:
df_ytd_cerrado3.to_csv('archivos_respaldo/df_ytd_cerrado3.csv', index = False)

### 4.2-1 - MERGE PARA -> YTD AÑO EN CURSO HASTA MES CERRADO (PERSPECTIVA MES ANTERIOR)
MERGE ENTRE df_libro_mayor_ytd_closed_2M y df_cod_proyecto

In [771]:
print("Dimensiones antes del merge:")
df_libro_mayor_ytd_closed_2M.shape

Dimensiones antes del merge:


(332, 3)

In [773]:
df_ytd_cerrado_2M = df_libro_mayor_ytd_closed_2M.copy()

In [774]:
# Se realiza merge con Nombre de Obra y Cliente.

print("Dimensiones df_ytd_cerrado_2M antes de cambios: ", df_ytd_cerrado_2M.shape)
df_ytd_cerrado_2M = df_ytd_cerrado_2M.merge(df_cod_proyecto, on='CODIGO_PROYECTO', how='left')
print("Dimensiones df_ytd_cerrado_2M antes de cambios: ", df_ytd_cerrado_2M.shape)
df_ytd_cerrado_2M.sample(10)

Dimensiones df_ytd_cerrado_2M antes de cambios:  (332, 3)
Dimensiones df_ytd_cerrado_2M antes de cambios:  (332, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
227,P-1699,INGRESO,73159443,Villa Panamericana Lote B,Empresa Constructora DLP S.A.
144,P-1679,COSTO MANO DE OBRA,-187507,Juan Sabaj,Constructora Grevia
195,P-1691,INGRESO,2124157,Rosario Rosales,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.
201,P-1693,COSTO MATERIALES,103024,Volta V0800,ECOBIO SPA
131,P-1674,INGRESO,0,Oficinas Beiersdorf Chorrillos,Constructora Suksa S.A.
276,P-1715,COSTO MANO DE OBRA,-12758158,CD Lo Echevers,Quezada y Boetsch
270,P-1713,COSTO SUPERVISIÓN,0,Reserva San Francisco,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...
100,P-1666,COSTO MANO DE OBRA,0,CD Axionlong,EBCO S.A.
238,P-1703,COSTO SUPERVISIÓN,0,Santa Victoria Danacorp,DANACORP S.A
16,P-1603,COSTO MANO DE OBRA,0,Farellones,Empresa Constructora DLP S.A.


In [775]:
# Se reemplazan nombres de TIPO_CUENTA

df_ytd_cerrado_2M['TIPO_CUENTA'] = df_ytd_cerrado_2M['TIPO_CUENTA'].replace({
    'INGRESO': '1. INGRESO',
    'COSTO MATERIALES': '2. COSTO MATERIALES',
    'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA',
    'COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'
})
df_ytd_cerrado_2M.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
68,P-1649,3. COSTO MANO DE OBRA,0,Inacap Vitacura E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.
139,P-1676,1. INGRESO,0,Lo Campino,EBCO S.A.
169,P-1685,2. COSTO MATERIALES,88900,Mapa Edificios No Operacionales,Claro Vicuña Valenzuela S.A.
17,P-1603,2. COSTO MATERIALES,-30000,Farellones,Empresa Constructora DLP S.A.
228,P-1700,3. COSTO MANO DE OBRA,-4264241,CD Orizon,EBCO S.A.
265,P-1712,2. COSTO MATERIALES,-13607471,Centro Comercial Viña Outlet Park,EBCO S.A.
209,P-1695,2. COSTO MATERIALES,-10303848,Edificio Estoril,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO
324,P-1727,3. COSTO MANO DE OBRA,-4714680,NaN,NaN
20,P-1611,3. COSTO MANO DE OBRA,0,Edificio Plaza Egaña,EBCO S.A.
295,P-1719,1. INGRESO,131348,CD METALPAR,EBCO S.A.


In [776]:
# INSPECCIÓN VISUAL 1 
df_ytd_cerrado_2M[df_ytd_cerrado_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-363774250,NaN,NaN
1,1. TOTAL,2. COSTO MATERIALES,-435882635,NaN,NaN
2,1. TOTAL,4. COSTO SUPERVISIÓN,-17923405,NaN,NaN
3,1. TOTAL,1. INGRESO,1118563896,NaN,NaN


In [777]:
#Se renombra "TOTAL" a "1. TOTAL" para que se pueda visualizar en el dashboard.

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
df_ytd_cerrado_2M[cols_to_replace] = df_ytd_cerrado_2M[cols_to_replace].replace('TOTAL', '1. TOTAL')
mask = df_ytd_cerrado['CODIGO_PROYECTO'] == '1. TOTAL'
df_ytd_cerrado_2M.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'
df_ytd_cerrado_2M.sample(4)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
244,P-1705,3. COSTO MANO DE OBRA,-8357637,Los Clarines,CONSTRUCTORA INARCO S.A.
320,P-1726,3. COSTO MANO DE OBRA,-2654140,NaN,NaN
105,P-1667,2. COSTO MATERIALES,-2362317,Eco Placilla,Constructora ICF SPA
331,P-1728,1. INGRESO,0,NaN,NaN


In [778]:
# INSPECCIÓN VISUAL 1
df_ytd_cerrado_2M[df_ytd_cerrado_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-363774250,1. TOTAL,1. TOTAL
1,1. TOTAL,2. COSTO MATERIALES,-435882635,1. TOTAL,1. TOTAL
2,1. TOTAL,4. COSTO SUPERVISIÓN,-17923405,1. TOTAL,1. TOTAL
3,1. TOTAL,1. INGRESO,1118563896,1. TOTAL,1. TOTAL


#### 4.2-1.1 - SE REALIZA CRUCE CON "df_libro_mayor5_base2" PARA CREAR DATAFRAME FINAL

In [779]:
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M   : ", df_ytd_cerrado_2M.shape)

df_ytd_cerrado_2M_2 = df_ytd_cerrado_2M.filter(["CODIGO_PROYECTO", "TIPO_CUENTA", "TOTAL"])

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_2 : ", df_ytd_cerrado_2M_2.shape)

df_ytd_cerrado_2M_2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M   :  (332, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_2 :  (332, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
237,P-1703,2. COSTO MATERIALES,-12417197
281,P-1716,2. COSTO MATERIALES,-23165972
325,P-1727,2. COSTO MATERIALES,-3753865
207,P-1694,1. INGRESO,36274900
202,P-1693,4. COSTO SUPERVISIÓN,0


In [780]:
df_margen[df_margen["CODIGO_PROYECTO"]=="P-1724"]

,CODIGO_PROYECTO,MARGEN
78,P-1724,0.72


In [781]:
# Exploración dataframe base

df_libro_mayor5_base2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
218,P-1656,3. COSTO MANO DE OBRA,0,MIRADOR CLUB HÍPICO,CONSTRUCTORA VITAL EDIFICACION SPA
149,P-1723,1. INGRESO,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.
230,P-1663,3. COSTO MANO DE OBRA,0,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA
128,P-1714,3. COSTO MANO DE OBRA,0,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA
82,P-1695,2. COSTO MATERIALES,0,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO


In [782]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

df_ytd_cerrado_2M_3 = df_libro_mayor5_base2.merge(df_ytd_cerrado_2M_2, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3       : ", df_ytd_cerrado_2M_3.shape)

df_ytd_cerrado_2M_3.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (332, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3       :  (332, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
95,P-1700,3. COSTO MANO DE OBRA,0,CD ORIZON,EBCO S.A.,-4264241
57,P-1686,3. COSTO MANO DE OBRA,0,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,-12899988
4,P-1611,2. COSTO MATERIALES,0,EDIFICIO PLAZA EGAÑA,EBCO S.A.,6648
290,P-1698,3. COSTO MANO DE OBRA,0,CD ANDES,EBCO S.A.,0
253,P-1674,3. COSTO MANO DE OBRA,0,OFICINAS BEIERSDORF CHORRILLOS,CONSTRUCTORA SUKSA S.A.,0


In [784]:
# INSPECCIÓN VISUAL 1

df_ytd_cerrado_2M_3[df_ytd_cerrado_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
328,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-17923405
329,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1118563896
330,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-435882635
331,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-363774250


#### 4.2-1.2 - SE REALIZA CREACIÓN DE COLUMNA MARGEN

In [785]:
# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ytd_cerrado_2M_3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES','3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_margen       : ", df_margen.shape)

# mostrar el DataFrame resultante
df_margen.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_margen       :  (83, 2)


,CODIGO_PROYECTO,MARGEN
26,P-1667,0.00
56,P-1699,0.44
49,P-1692,0.08
78,P-1724,0.72
47,P-1690,0.35
46,P-1689,0.00
81,P-1727,0.23
25,P-1666,0.00
11,P-1641,0.00
63,P-1707,-0.60


In [786]:
df_margen[df_margen["CODIGO_PROYECTO"]=="P-1724"]

,CODIGO_PROYECTO,MARGEN
78,P-1724,0.72


In [787]:
# Merge Dataframe para agregar MARGEN

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3   : ", df_ytd_cerrado_2M_3.shape)

df_ytd_cerrado_2M_3 = df_ytd_cerrado_2M_3.merge(df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3 : ", df_ytd_cerrado_2M_3.shape)

df_ytd_cerrado_2M_3.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3   :  (332, 6)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3 :  (332, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
147,P-1723,3. COSTO MANO DE OBRA,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-10269821,-0.16
168,P-1591,2. COSTO MATERIALES,0,EDIFICIO ESTORIL,EBCO S.A.,0,0.00
158,P-1726,1. INGRESO,0,NaN,NaN,21037279,0.19
308,P-1717,3. COSTO MANO DE OBRA,0,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0,0.00
231,P-1663,4. COSTO SUPERVISIÓN,0,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA,0,0.00


In [788]:
df_ytd_cerrado_2M_3.to_csv('archivos_respaldo/df_ytd_cerrado_2M_3.csv', index = False)

### 4.3 - MERGE MES EN CURSO ENTRE LIBRO MAYOR + PROYECTADO 
(DATAFRAMES: df_libro_mayor5_mec2 y df_proyectado4)

## 5. - EXPORTACIÓN DE ARCHIVOS

In [746]:
# Conexión a google drive

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# Credenciales de GOOGLE API, en archivo .json
credentials = ServiceAccountCredentials.from_json_keyfile_name('keen-extension-358919-9214486a06be.json', scope) # Your json file here

gc = gspread.authorize(credentials)

## SIEMPRE DAR ACCESO A ESTE USUARIO: test-321@keen-extension-358919.iam.gserviceaccount.com

### 5.1 EXPORTACIÓN LIBRO MAYOR PARA CUADRO FINANCIERO DE PROYECTOS

In [748]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1COX0_p3dTPNLZMwoh__0uQdOYfP_AA1mw7lfKx4SMMQ')

# select a work sheet from its name
worksheet1 = gs.worksheet('Hoja 1')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=df_libro_mayor6, include_index=False,
include_column_header=True, resize=True)

### 5.2 EXPORTACIÓN LIBRO MAYOR YTD HASTA MES CERRADO (MES ANTERIOR)
Dataframe: df_ytd_cerrado3

In [770]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1_53Vj0j4xnqsWDdHDdetp3JmoUHRjLlvsUyi7tQXCY4')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ytd_cerrado3, include_index=False,
include_column_header=True, resize=True)

### 5.2 EXPORTACIÓN LIBRO MAYOR YTD HASTA MES CERRADO (MES ANTERIOR) PERSPECTIVA MES ANTERIOR
Dataframe: df_ytd_cerrado_2M_3

In [790]:
gs = gc.open_by_key('1_53Vj0j4xnqsWDdHDdetp3JmoUHRjLlvsUyi7tQXCY4')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ytd_cerrado_2M_3, include_index=False,
include_column_header=True, resize=True)